In [3]:
import os
import json
import xmltodict
from glob import glob
from tqdm import tqdm

In [ ]:
# xml 파일의 키포인트, json 파일의 라벨을 합쳐 merged_keypoint 생성

# 📁 경로 설정 (Lightning 기준)
base_xml_root = 'training/keypoint/tact_keypoints/FIRE'
base_json_root = 'training/keypoint/tact_morpheme/FIRE'
save_dir = 'training/merged_keypoint'
os.makedirs(save_dir, exist_ok=True)

# 상체 & 손 중심 keypoint 인덱스
POSE_KEEP_IDX = [0, 1, 2, 3, 4, 5, 6, 7, 15, 16, 17, 18]

# 2D pose keypoint 필터링
def filter_pose_keypoints(points):
    keypoints = [list(map(float, p.split(','))) for p in points]
    return [coord for idx in POSE_KEEP_IDX if idx < len(keypoints) for coord in keypoints[idx][:2]]

# hand keypoint 필터링
def filter_hand_keypoints(points):
    keypoints = [list(map(float, p.split(','))) for p in points]
    return [coord for kpt in keypoints for coord in kpt[:2]]

# 단일 샘플 병합 함수
def merge_single_sample_from_path(xml_path, json_path):
    try:
        with open(xml_path, 'r', encoding='utf-8') as f:
            xml_data = xmltodict.parse(f.read())
    except Exception:
        print(f"[오류] XML 파싱 실패: {xml_path}")
        return False

    frames_dict = {}
    for track in xml_data["annotations"]["track"]:
        label = track["@label"]
        if label == "face_keypoints_2d":
            continue
        for key in track:
            if key not in ["@id", "@label"]:
                entries = track[key] if isinstance(track[key], list) else [track[key]]
                for entry in entries:
                    idx_f = int(entry["@frame"])
                    points = [p for p in entry["@points"].split(';') if p.strip()]
                    filtered_points = (
                        filter_pose_keypoints(points) if label == "pose_keypoints_2d"
                        else filter_hand_keypoints(points)
                    )
                    frames_dict.setdefault(idx_f, {})[label] = filtered_points

    try:
        with open(json_path, "r", encoding="utf-8") as f:
            js = json.load(f)
        korean_text = js.get("korean_text", "")
        signer_id = js.get("translator", {}).get("id", None)
    except Exception:
        print(f"[오류] JSON 파싱 실패: {json_path}")
        return False

    base = os.path.splitext(os.path.basename(xml_path))[0]
    result = {
        "index": base,
        "id": signer_id,
        "korean_text": korean_text,
        "frames": []
    }

    prev_frame_vec = None
    for idx_f in sorted(frames_dict):
        frame_info = frames_dict[idx_f]
        frame_vec = frame_info.get("pose_keypoints_2d", []) + \
                    frame_info.get("hand_left_keypoints_2d", []) + \
                    frame_info.get("hand_right_keypoints_2d", [])
        if prev_frame_vec is not None and frame_vec == prev_frame_vec:
            continue
        prev_frame_vec = frame_vec
        result["frames"].append({
            "frame_idx": idx_f,
            "pose": frame_info.get("pose_keypoints_2d", []),
            "hand_left": frame_info.get("hand_left_keypoints_2d", []),
            "hand_right": frame_info.get("hand_right_keypoints_2d", [])
        })

    save_path = os.path.join(save_dir, f"{base}.json")
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    return True

# 전체 xml/json 병합
xml_paths = glob(os.path.join(base_xml_root, "**", "*.xml"), recursive=True)

success, fail = 0, 0
for xml_path in tqdm(xml_paths, desc="병합 진행 중"):
    base_name = os.path.basename(xml_path)
    json_name = base_name.replace("_F.xml", ".json")
    relative_dir = os.path.relpath(os.path.dirname(xml_path), base_xml_root)
    json_path = os.path.join(base_json_root, relative_dir, json_name)

    if os.path.exists(json_path):
        ok = merge_single_sample_from_path(xml_path, json_path)
        success += int(ok)
        fail += int(not ok)
    else:
        print(f"❗ 대응 JSON 없음: {json_path}")
        fail += 1

print(f"\n✅ 병합 완료: {success}개, 실패: {fail}개")

In [ ]:
# xml, json 간 mapping 확인

from glob import glob
import os

# XML / JSON 경로
xml_root = 'training/keypoint/tact_keypoints/FIRE'
json_root = 'training/keypoint/tact_morpheme/FIRE'

# 추출 함수
def extract_key_from_filename(path, suffix_to_strip):
    base = os.path.basename(path)
    key = base.replace(suffix_to_strip, '')  # 예: '_F.xml' 또는 '.json'
    return key

# 목록 수집
xml_paths = glob(os.path.join(xml_root, "**", "*.xml"), recursive=True)
json_paths = glob(os.path.join(json_root, "**", "*.json"), recursive=True)

xml_keys = set(extract_key_from_filename(p, '_F.xml') for p in xml_paths)
json_keys = set(extract_key_from_filename(p, '.json') for p in json_paths)

# 교집합, 차집합 확인
only_in_xml = sorted(xml_keys - json_keys)
only_in_json = sorted(json_keys - xml_keys)
matched = sorted(xml_keys & json_keys)

# 결과 출력
print(f"📊 총 XML 파일: {len(xml_keys)}")
print(f"📊 총 JSON 파일: {len(json_keys)}")
print(f"✅ 매칭된 파일 수: {len(matched)}")
print(f"❗ XML만 있고 JSON 없는 파일 수: {len(only_in_xml)}")
print(f"❗ JSON만 있고 XML 없는 파일 수: {len(only_in_json)}")

# 샘플 출력
if only_in_xml:
    print("\n📁 XML만 있는 파일 예시:", only_in_xml[:5])
if only_in_json:
    print("\n📁 JSON만 있는 파일 예시:", only_in_json[:5])


In [8]:
# Merged_keypoint 파일 확인

# 실제 병합된 JSON 경로로 수정
sample_path = 'training/train/norm_keypoint/NIA_SL_G2_FIRE000003_1_TW03_F_norm.json'

# JSON 파일 열기
with open(sample_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 병합된 구조 정보 출력
print("Top-level keys:", list(data.keys()))
print("Sample index:", data['index'])
print("Signer ID:", data.get('id'))
print("Korean text:", data['korean_text'])
print("Total frames:", len(data['frames']))
print("First frame example:\n", data['frames'][0])

# 키포인트 확인 (예: pose = 12 keypoints × (x, y) = 24차원)
pose_len = len(data['frames'][1]['pose'])
print(f"\n Frame 1 pose keypoints vector length: {pose_len} (expected 24)")

Top-level keys: ['index', 'id', 'korean_text', 'frames', 'masked_korean_text', 'slot_values']
Sample index: NIA_SL_G2_FIRE000003_1_TW03_F
Signer ID: None
Korean text: 안전안내. 오늘 02:00 반월공단 내 화재 발생, 연기확산에 주의하세요
Total frames: 415
First frame example:
 {'frame_idx': 0, 'pose': [-0.5705703748867382, -1.293855761361989, -0.9978378664332288, 0.38877772926518217, -1.333126966363792, 0.4141644645741816, -0.19592469723364872, 1.0743982394172575, -0.6269272316283744, 1.9253424647875002, 0.09971286732763626, 0.22130937010234883, -0.8861206474747211, 0.9415592908037219, 0.06099393017401008, 1.0303200784063804, -0.4102040536331944, -1.2634992735993633, -0.4574661803063296, -1.1997975506862708, -0.3532081551955483, -0.9235349309592826, -0.25358581902873323, -1.2418483940684855], 'hand_left': [0.2562509398573034, 0.3781380067836563, 0.23828920569760192, 0.40224842894696955, 0.22913406363610123, 0.44531825872919373, 0.2135864074856869, 0.44406526391357304, 0.19717924048813995, 0.44449387108595795, 0.243

In [8]:
# 문장 수 세기

import os
from collections import defaultdict

DATA_DIR = "/teamspace/studios/this_studio/training/merged_keypoint"
file_list = [f for f in os.listdir(DATA_DIR) if f.endswith(".json")]

sentence_to_files = defaultdict(list)

for fname in file_list:
    # 예: NIA_SL_G2_FIRE000004_1_KU02_F.json
    parts = fname.split("_")
    if len(parts) >= 4:
        sentence_id = parts[3]  # FIRE000004
        sentence_to_files[sentence_id].append(fname)

print("총 문장 ID 수:", len(sentence_to_files))

총 문장 ID 수: 1828


In [9]:
# Train, val split

import os
import random
import shutil
from tqdm import tqdm

# 경로 설정
SOURCE_DIR = "/teamspace/studios/this_studio/training/merged_keypoint"
TRAIN_DIR = "/teamspace/studios/this_studio/training/train/merged_keypoint"
VAL_DIR = "/teamspace/studios/this_studio/training/val/merged_keypoint"

os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(VAL_DIR, exist_ok=True)

# 모든 json 파일 수집
all_files = [f for f in os.listdir(SOURCE_DIR) if f.endswith(".json")]

# 문장 ID 추출 (예: NIA_SL_G2_FIRE000004_1_KU02_F.json → FIRE000004)
sentence_to_files = dict()
for fname in all_files:
    parts = fname.split("_")
    if len(parts) >= 4:
        sentence_id = parts[3]  # 올바른 문장 ID
        sentence_to_files.setdefault(sentence_id, []).append(fname)

# 문장 ID 중에서 200개 랜덤 선택 (검증용)
all_sentence_ids = list(sentence_to_files.keys())
val_sentence_ids = set(random.sample(all_sentence_ids, 200))

# 파일 분배
for sid, fnames in tqdm(sentence_to_files.items(), desc="Copying files"):
    target_dir = VAL_DIR if sid in val_sentence_ids else TRAIN_DIR
    for fname in fnames:
        src = os.path.join(SOURCE_DIR, fname)
        dst = os.path.join(target_dir, fname)
        shutil.copy2(src, dst)

print(f"✅ 분리 완료! 총 문장 수: {len(all_sentence_ids)} / Validation 문장 수: {len(val_sentence_ids)}")


Copying files: 100%|██████████| 1828/1828 [00:20<00:00, 90.94it/s] 

✅ 분리 완료! 총 문장 수: 1828 / Validation 문장 수: 200


In [2]:
# Train, Val 파일 수 확인

import os

TRAIN_DIR = "/teamspace/studios/this_studio/training/train/merged_keypoint"
VAL_DIR = "/teamspace/studios/this_studio/training/val/merged_keypoint"

num_train = len([f for f in os.listdir(TRAIN_DIR) if f.endswith(".json")])
num_val = len([f for f in os.listdir(VAL_DIR) if f.endswith(".json")])

print(f"📂 Train JSON 파일 수: {num_train}")
print(f"📂 Validation JSON 파일 수: {num_val}")


📂 Train JSON 파일 수: 3183
📂 Validation JSON 파일 수: 408


In [11]:
# Train, Val 각각 normalization 적용 

import os
import json
import numpy as np
from glob import glob
from tqdm import tqdm

# 경로 설정
TRAIN_SRC = "/teamspace/studios/this_studio/training/train/merged_keypoint"
VAL_SRC = "/teamspace/studios/this_studio/training/val/merged_keypoint"
TRAIN_DST = "/teamspace/studios/this_studio/training/train/norm_keypoint"
VAL_DST = "/teamspace/studios/this_studio/training/val/norm_keypoint"
STAT_PATH = "/teamspace/studios/this_studio/training/norm_stat.npz"

# 디렉토리 생성
os.makedirs(TRAIN_DST, exist_ok=True)
os.makedirs(VAL_DST, exist_ok=True)

### 1. 통계 계산용 데이터 수집 (Train 기준)
pose_all, left_all, right_all = [], [], []

train_files = sorted(glob(os.path.join(TRAIN_SRC, "*.json")))
for file in tqdm(train_files, desc="🔍 Collecting train keypoints"):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    for frame in data['frames']:
        if frame.get('pose'): pose_all.append(frame['pose'])
        if frame.get('hand_left'): left_all.append(frame['hand_left'])
        if frame.get('hand_right'): right_all.append(frame['hand_right'])

pose_all = np.array(pose_all)
left_all = np.array(left_all)
right_all = np.array(right_all)

# 통계 계산 및 저장
pose_mu = pose_all.mean(axis=0)
pose_sd = pose_all.std(axis=0)
left_min = left_all.min(axis=0)
left_max = left_all.max(axis=0)
right_min = right_all.min(axis=0)
right_max = right_all.max(axis=0)

np.savez(STAT_PATH, pose_mu=pose_mu, pose_sd=pose_sd,
         left_min=left_min, left_max=left_max,
         right_min=right_min, right_max=right_max)
print(f"📊 Train 기반 정규화 통계 저장 완료 → {STAT_PATH}")

### 2. 정규화 함수 정의
def normalize_pose(x, mu, sd):
    return (np.array(x) - mu) / (sd + 1e-8)

def normalize_hand(x, minv, maxv):
    return (np.array(x) - minv) / (maxv - minv + 1e-8) - 0.5

### 3. 정규화 적용 함수
def apply_normalization_and_save(src_files, dst_dir, stats):
    for file in tqdm(src_files, desc=f"🌀 Normalizing → {dst_dir}"):
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        for frame in data['frames']:
            if frame.get('pose'):
                frame['pose'] = list(normalize_pose(frame['pose'], stats['pose_mu'], stats['pose_sd']))
            if frame.get('hand_left'):
                frame['hand_left'] = list(normalize_hand(frame['hand_left'], stats['left_min'], stats['left_max']))
            if frame.get('hand_right'):
                frame['hand_right'] = list(normalize_hand(frame['hand_right'], stats['right_min'], stats['right_max']))
        fname = os.path.basename(file).replace('.json', '_norm.json')
        with open(os.path.join(dst_dir, fname), 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

### 4. 정규화 실행
stats = np.load(STAT_PATH)

# Train 정규화
apply_normalization_and_save(train_files, TRAIN_DST, stats)

# Val 정규화 (stat은 train 기준)
val_files = sorted(glob(os.path.join(VAL_SRC, "*.json")))
apply_normalization_and_save(val_files, VAL_DST, stats)

print("✅ Train & Validation 정규화 완료!")


🔍 Collecting train keypoints: 100%|██████████| 3183/3183 [00:41<00:00, 77.32it/s] 


📊 Train 기반 정규화 통계 저장 완료 → /teamspace/studios/this_studio/training/norm_stat.npz


🌀 Normalizing → /teamspace/studios/this_studio/training/train/norm_keypoint: 100%|██████████| 3183/3183 [14:25<00:00,  3.68it/s]
🌀 Normalizing → /teamspace/studios/this_studio/training/val/norm_keypoint: 100%|██████████| 408/408 [01:52<00:00,  3.63it/s]

✅ Train & Validation 정규화 완료!


In [12]:
import os
import json
import numpy as np
from glob import glob
from tqdm import tqdm

# 정규화된 데이터 경로
NORM_TRAIN_DIR = "/teamspace/studios/this_studio/training/train/norm_keypoint"

# 누적 저장
pose_all, left_all, right_all = [], [], []

# 모든 정규화된 train json 읽기
norm_files = sorted(glob(os.path.join(NORM_TRAIN_DIR, "*.json")))
for file in tqdm(norm_files, desc="🔍 Checking normalized data"):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    for frame in data['frames']:
        if frame.get('pose'): pose_all.append(frame['pose'])
        if frame.get('hand_left'): left_all.append(frame['hand_left'])
        if frame.get('hand_right'): right_all.append(frame['hand_right'])

# numpy 변환
pose_all = np.array(pose_all)
left_all = np.array(left_all)
right_all = np.array(right_all)

# 결과 출력
print("✅ Normalized Pose:")
print("  mean:", np.round(pose_all.mean(axis=0)[:5], 4))
print("  std: ", np.round(pose_all.std(axis=0)[:5], 4))

print("\n✅ Normalized Hand Left:")
print("  min:", np.round(left_all.min(axis=0)[:5], 4))
print("  max:", np.round(left_all.max(axis=0)[:5], 4))

print("\n✅ Normalized Hand Right:")
print("  min:", np.round(right_all.min(axis=0)[:5], 4))
print("  max:", np.round(right_all.max(axis=0)[:5], 4))


🔍 Checking normalized data: 100%|██████████| 3183/3183 [01:35<00:00, 33.28it/s]


✅ Normalized Pose:
  mean: [ 0.  0.  0. -0.  0.]
  std:  [1. 1. 1. 1. 1.]

✅ Normalized Hand Left:
  min: [-0.5 -0.5 -0.5 -0.5 -0.5]
  max: [0.5 0.5 0.5 0.5 0.5]

✅ Normalized Hand Right:
  min: [-0.5 -0.5 -0.5 -0.5 -0.5]
  max: [0.5 0.5 0.5 0.5 0.5]


In [19]:
# Data Augmentation via Skip Sampling 적용 (중복된 frame 출력)

import os
import json
import random
import numpy as np
import copy
from glob import glob
from tqdm import tqdm

# 설정
NORM_DIR = "/teamspace/studios/this_studio/training/train/norm_keypoint"
AUG_DIR = "/teamspace/studios/this_studio/training/train/augmented_keypoint"
os.makedirs(AUG_DIR, exist_ok=True)

AUG_FACTOR = 50
N_FRAMES_SAMPLED = 50

file_list = sorted(glob(os.path.join(NORM_DIR, "*_norm.json")))

# 중복이 발생한 augmentation 수
duplicate_aug_count = 0

def safe_sampling_indices(l, n):
    z = int(np.floor(l / (n - 1)))
    y = int(np.floor((l - z * (n - 1)) / 2))
    baseline_idx = [y + z * k for k in range(n)]

    # soft clipping
    if baseline_idx[-1] >= l:
        baseline_idx[-1] = l - 1

    return baseline_idx, z

for src_path in tqdm(file_list, desc="🔁 Skip Sampling Augmentation (train only)"):
    basename = os.path.basename(src_path)
    file_idx = basename.replace('_norm.json', '')

    with open(src_path, 'r', encoding='utf-8') as f:
        src_data = json.load(f)

    frames = src_data['frames']
    l = len(frames)
    n = N_FRAMES_SAMPLED

    if l < n:
        # 길이 부족 시 패딩
        indices = list(range(l)) + [l - 1] * (n - l)
        sampled_frames = [frames[i] for i in indices]
        new_data = copy.deepcopy(src_data)
        new_data['frames'] = sampled_frames
        for i, frame in enumerate(new_data['frames']):
            frame['frame_idx'] = i
        save_path = os.path.join(AUG_DIR, f"augmented_{file_idx}_1.json")
        with open(save_path, 'w', encoding='utf-8') as wf:
            json.dump(new_data, wf, ensure_ascii=False, indent=2)
    else:
        baseline_idx, z = safe_sampling_indices(l, n)

        if z == 1:
            indices = [min(l - 1, max(0, idx)) for idx in baseline_idx]
            sampled_frames = [frames[i] for i in indices]
            new_data = copy.deepcopy(src_data)
            new_data['frames'] = sampled_frames
            for i, frame in enumerate(new_data['frames']):
                frame['frame_idx'] = i
            save_path = os.path.join(AUG_DIR, f"augmented_{file_idx}_1.json")
            with open(save_path, 'w', encoding='utf-8') as wf:
                json.dump(new_data, wf, ensure_ascii=False, indent=2)
        else:
            for aug_idx in range(1, AUG_FACTOR + 1):
                indices = []
                for k in range(n):
                    base = baseline_idx[k]
                    if base >= l - 1:
                        noise = 0
                    else:
                        noise_max = max(1, l - 1 - base) if k == n - 1 else max(1, z)
                        noise = random.randint(0, noise_max - 1)
                    idx = min(base + noise, l - 1)
                    indices.append(idx)

                # ✅ 중복 여부 확인
                if len(set(indices)) < len(indices):
                    duplicate_aug_count += 1

                sampled_frames = [frames[i] for i in indices]
                new_data = copy.deepcopy(src_data)
                new_data['frames'] = sampled_frames
                for i, frame in enumerate(new_data['frames']):
                    frame['frame_idx'] = i
                save_path = os.path.join(AUG_DIR, f"augmented_{file_idx}_{aug_idx}.json")
                with open(save_path, 'w', encoding='utf-8') as wf:
                    json.dump(new_data, wf, ensure_ascii=False, indent=2)

# 🔚 최종 결과 출력
print(f"\n✅ 모든 train 파일에서 augmentation 결과를 '{AUG_DIR}'에 저장했습니다.")
print(f"🔍 중복된 frame이 포함된 augmentation 수: {duplicate_aug_count:,}")

🔁 Skip Sampling Augmentation (train only): 100%|██████████| 3183/3183 [1:05:35<00:00,  1.24s/it]


✅ 모든 train 파일에서 augmentation 결과를 '/teamspace/studios/this_studio/training/train/augmented_keypoint'에 저장했습니다.
🔍 중복된 frame이 포함된 augmentation 수: 288


In [4]:
# 50개의 프레임은 너무 작은 거 아닐까? - 100 프레임으로 늘리는 것 고려
# 학습 자체가 어려울 수 있음 (고유명사가 너무 많고 데이터가 부족)

import os
import json
from glob import glob
import random

# 🔧 경로 설정: train 기준
MERGED_DIR = "/teamspace/studios/this_studio/training/train/merged_keypoint"  # 또는 norm_keypoint
file_list = sorted(glob(os.path.join(MERGED_DIR, "*.json")))

# 🔀 무작위 50개 샘플 추출
sample_files = random.sample(file_list, 50)

# 🔍 korean_text 추출
texts = []
for path in sample_files:
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        text = data.get("korean_text", "").strip()
        if text:  # 빈 문장 제외
            texts.append(text)

# 📤 출력
print("📚 Korean Text Samples (max 50):\n")
for i, t in enumerate(texts, 1):
    print(f"{i:02d}. {t}")


📚 Korean Text Samples (max 50):

01. 화재관련, 중구 북성동에서 화재가 발생하여 진압중이니 인근 주민들께서는 안전에 유의하시기 바랍니다.
02. 7.8 07:37 구포동 516번지 일원 단독주택 화재 발생으로 일대가 혼잡하오니 주민들께서는 외출을 자제하시기 바랍니다.
03. 금일 09:30 고양시 일산서구 덕이동 177-19번지 인근에 화재 발생. 이 지역을 우회하여 주시고 인근 주민은 안전사고 발생에 유의 바랍니다.
04. 오늘 09:17 대전시 동구 상소동 628-22번지 인근 단독주택 화재 발생. 인근 주민은 안전한 곳으로 대피하고 차량은 우회 바랍니다.
05. 오늘 09:00 우장산 스포츠센터 화재 발생. 인근 주민은 안전한 곳으로 대피하여 주시기 바랍니다.
06. 오늘 09:30 남원시 사매2터널 인근 전주폐차장에서 화재 발생, 인근주민께서는 화재 등 안전에 유의하시기 바랍니다.
07. 오늘 12:40 오포읍 문형리 908-36번지 대형화재 발생. 이 지역을 우회하여 주시고 인근 주민은 안전사고 발생에 유의 바랍니다.
08. 동점→석포 행 도로 탱크로리 화재 사고. 탱크로리 화재로 인하여 전면통제 하오니 우회하시기 바랍니다.
09. 오늘 14:50 남구 대명동 995-96번지 인근에 화재가 발생하였으니, 인근 주민은 안전사고에 주의하시기 바랍니다
10. 오늘 오후 7시부터 광명시 노온사동 비닐하우스에서 화재 발생, 인근 주민은 안전한 곳으로 대피 바랍니다.
11. 금일 09:02 광명시 노온사동 신대호수사거리 고가도로 화재 발생, 인근 주민은 안전한 곳으로 대피하시기 바랍니다.
12. 오늘 07:50 상동면 우계리 479-88 송유관공사 화재 발생으로 인근 주민은 안전사고에 유의하시기 바랍니다.
13. 04시 15분경 오포읍 문형리 312-79번지 오피스텔 화재 발생. 인근 주민은 안전한 곳으로 대피하시기 바랍니다.
14. 오늘 16:20 석남동 840-21 화재 발생. 주변으로 확산될 우려가 있으니 인근 주민은 대피 바랍

In [3]:
END_PATTERNS = [
    "바랍니다", "주십시오", "주세요", "주시오",
    "권고드립니다", "안내드립니다", "알려드립니다",
    "하십시오", "하십시오.", "해 주십시오", "해 주세요"
]

def extract_expanded_endings(json_dir: str, field="korean_text", top_n=50):
    from collections import Counter
    from pathlib import Path
    import json

    counter = Counter()

    for file in Path(json_dir).glob("*.json"):
        with open(file, encoding="utf-8") as f:
            text = json.load(f).get(field, "").strip()

        for n in range(4, 20):  # 최대 20글자까지 뒤에서 잘라봄
            candidate = text[-n:]
            if any(p in candidate for p in END_PATTERNS):
                counter[candidate] += 1

    return counter.most_common(top_n)


In [5]:
top_endings = extract_expanded_endings("training/train/norm_keypoint")
for phrase, count in top_endings:
    print(f"{phrase}: {count}")

바랍니다.: 2892
 바랍니다.: 2664
기 바랍니다.: 1917
시기 바랍니다.: 1832
하시기 바랍니다.: 1411
의하시기 바랍니다.: 849
유의하시기 바랍니다.: 560
 유의하시기 바랍니다.: 560
에 유의하시기 바랍니다.: 559
의 바랍니다.: 554
유의 바랍니다.: 429
 유의 바랍니다.: 429
에 유의 바랍니다.: 429
주시기 바랍니다.: 421
 주시기 바랍니다.: 405
생에 유의 바랍니다.: 331
발생에 유의 바랍니다.: 331
 발생에 유의 바랍니다.: 331
고 발생에 유의 바랍니다.: 331
사고 발생에 유의 바랍니다.: 331
전사고 발생에 유의 바랍니다.: 331
안전사고 발생에 유의 바랍니다.: 331
 안전사고 발생에 유의 바랍니다.: 331
은 안전사고 발생에 유의 바랍니다.: 320
피하시기 바랍니다.: 314
대피하시기 바랍니다.: 314
 대피하시기 바랍니다.: 314
전에 유의하시기 바랍니다.: 294
안전에 유의하시기 바랍니다.: 294
 안전에 유의하시기 바랍니다.: 291
주의하시기 바랍니다.: 289
 주의하시기 바랍니다.: 289
로 대피하시기 바랍니다.: 268
여 주시기 바랍니다.: 264
하여 주시기 바랍니다.: 264
으로 대피하시기 바랍니다.: 262
곳으로 대피하시기 바랍니다.: 253
 곳으로 대피하시기 바랍니다.: 253
한 곳으로 대피하시기 바랍니다.: 253
전한 곳으로 대피하시기 바랍니다.: 253
안전한 곳으로 대피하시기 바랍니다.: 253
에 주의하시기 바랍니다.: 252
생에 유의하시기 바랍니다.: 203
발생에 유의하시기 바랍니다.: 203
 발생에 유의하시기 바랍니다.: 200
고 발생에 유의하시기 바랍니다.: 200
사고 발생에 유의하시기 바랍니다.: 200
전사고 발생에 유의하시기 바랍니다.: 200
회하여 주시기 바랍니다.: 166
우회하여 주시기 바랍니다.: 166


In [13]:
import json
from pathlib import Path

# ✅ 유의 바랍니다로 정규화
NORMALIZATION_DICT = {
#    "주의하시기 바랍니다": "유의 바랍니다",
#    "유의하시기 바랍니다.": "유의 바랍니다",
#    "유의하시기 바랍니다": "유의 바랍니다", 
    "주의 바랍니다": "유의 바랍니다"
}

def normalize_text_fields(data: dict, norm_dict: dict) -> dict:
    for key in ["korean_text", "masked_korean_text"]:
        if key in data:
            for from_expr, to_expr in norm_dict.items():
                data[key] = data[key].replace(from_expr, to_expr)
    return data

def apply_normalization_to_dir(json_dir: Path):
    json_files = list(json_dir.glob("*.json"))
    print(f"Processing {len(json_files)} files in: {json_dir}")
    
    for file in json_files:
        with open(file, encoding="utf-8") as f:
            data = json.load(f)

        # 정규화 적용
        data = normalize_text_fields(data, NORMALIZATION_DICT)

        # 덮어쓰기 저장
        with open(file, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

# ✅ 적용 대상 디렉토리
train_dir = Path("training/train/norm_keypoint")
val_dir = Path("training/val/norm_keypoint")

# 🔁 정규화 실행
apply_normalization_to_dir(train_dir)
apply_normalization_to_dir(val_dir)

Processing 3183 files in: training/train/norm_keypoint
Processing 408 files in: training/val/norm_keypoint


In [51]:
from pathlib import Path
import json
import re
from tqdm import tqdm

def normalize_day_in_json(json_dir: str, field="korean_text"):
    files = list(Path(json_dir).glob("*.json"))

    for file in tqdm(files, desc=f"📂 Normalizing '{field}' in {json_dir}"):
        with open(file, encoding="utf-8") as f:
            data = json.load(f)

        if field in data:
            original = data[field]
            normalized = re.sub(r"(금일|당일)", "오늘", original)
            data[field] = normalized

        with open(file, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

train_dir = Path("training/train/norm_keypoint")
val_dir = Path("training/val/norm_keypoint")

normalize_day_in_json(train_dir)
normalize_day_in_json(val_dir)

📂 Normalizing 'korean_text' in training/train/norm_keypoint: 100%|██████████| 3183/3183 [04:13<00:00, 12.53it/s]
📂 Normalizing 'korean_text' in training/val/norm_keypoint: 100%|██████████| 408/408 [00:32<00:00, 12.48it/s]


In [14]:
import json
from pathlib import Path
from collections import Counter

# 확인하고 싶은 표현들 (정규화 전 표현 + 정규화 후 표현)
CHECK_PHRASES = [
    "주의하시기 바랍니다",
    "유의하시기 바랍니다",
    "유의하시기 바랍니다.",
    "주의 바랍니다",
    "유의 바랍니다"
]

def count_phrases_in_dir(json_dir: Path, check_phrases):
    counter = Counter()
    files = list(json_dir.glob("*.json"))

    for file in files:
        with open(file, encoding="utf-8") as f:
            data = json.load(f)

        for key in ["korean_text", "masked_korean_text"]:
            text = data.get(key, "")
            for phrase in check_phrases:
                if phrase in text:
                    counter[phrase] += 1

    print(f"📂 결과 (in {json_dir}):")
    for phrase in check_phrases:
        print(f"  🔎 '{phrase}': {counter[phrase]}회")

# ✅ 실행
train_dir = Path("training/train/norm_keypoint")
val_dir = Path("training/val/norm_keypoint")

count_phrases_in_dir(train_dir, CHECK_PHRASES)
count_phrases_in_dir(val_dir, CHECK_PHRASES)


📂 결과 (in training/train/norm_keypoint):
  🔎 '주의하시기 바랍니다': 0회
  🔎 '유의하시기 바랍니다': 0회
  🔎 '유의하시기 바랍니다.': 0회
  🔎 '주의 바랍니다': 0회
  🔎 '유의 바랍니다': 1437회
📂 결과 (in training/val/norm_keypoint):
  🔎 '주의하시기 바랍니다': 0회
  🔎 '유의하시기 바랍니다': 0회
  🔎 '유의하시기 바랍니다.': 0회
  🔎 '주의 바랍니다': 0회
  🔎 '유의 바랍니다': 174회


In [ ]:
# ✅ 가장 마지막에 등장하는 slot_value만 저장하는 방식으로 유지하며,
#    동작을 명확히 하기 위해 덮어쓰기 로직을 명시함 (기존에도 이 방식이었음)

import re
from typing import Tuple, Dict

def grouped_region_mask_korean_text_final(text: str) -> Tuple[str, Dict[str, str]]:
    slot_values = {}
    REGION_END = r'(?=\s|[.,은는이가의에에서와]|[0-9]|$)'
    JOSA = r"(은|는|이|가|에|에서|로|를|과|와|도|까|까지|만|으로|요|도)"
    EXCLUDE_PATTERN = r'\b[가-힣]{1,10}(사고|중|발생)\b'
    EXCLUDE_FOR_STATION = r'(사거리|삼거리|오거리|도로|고가도로|역사|하상도로|지하차도|차도|우회도로)'

    patterns = [
        (r'([0-2]?[0-9]:[0-5][0-9])', lambda m: [("<시간>", m.group(1))]),
        (r'([0-9]{1,2}시\s?[0-9]{0,2}분?)', lambda m: [("<시간>", m.group(1))]),
        # ✅ 먼저 복합 지역 단어 (산 들어간 지역 방어용)
        (r'\b[가-힣]{2,10}(시|군|구|서구|북구|남구|동구)\b', lambda m: [("<지역>", m.group(0))]),
    
        # ✅ 산 (예외 단어 방어 포함)
        (r'(?<!부산)(?<!확산)(?<!등산)(?<!우산)(?<!횡산)([가-힣]{1,10}산)(?=\s|[.,은는이가의에에서와]|[0-9]|$)', 
         lambda m: [("<산>", m.group(1))]),
        (r'([가-힣0-9]{1,20}(사거리|삼거리|오거리|육거리)?\s?(고가도로|하상도로|지하차도|도로|차도|우회도로))',
         lambda m: [("<도로>", m.group(0))]),
        (r'([가-힣]{1,20}(사거리|삼거리|오거리|육거리))',
         lambda m: [("<도로>", m.group(1))]),
        (rf'([가-힣]{{1,10}}(읍|면|동|리))\s([가-힣]{{1,10}}(리)){REGION_END}',
         lambda m: [("<지역>", f"{m.group(1)} {m.group(3)}")]),
        (rf'([가-힣]{{1,10}}(시|도|군|구))\s?([가-힣]{{1,10}}(동|읍|면|리)){REGION_END}',
         lambda m: [("<지역>", f"{m.group(1)} {m.group(3)}")]),
        (r'\b([가-힣]{1,10}(시|도|군|구|읍|면|동|리))\b', lambda m: [("<지역>", m.group(1))]),
        (r'([가-힣]{2,10}[0-9]{1,2}가)', lambda m: [("<주소>", m.group(1))]),
        (r'([0-9]{1,5}-[0-9]{1,5}(번지)?)', lambda m: [("<주소>", m.group(1))]),
        (r'([가-힣]{1,20}(로|길)\s?[0-9]{1,4}(-[0-9]{1,4})?(번지)?)', lambda m: [("<주소>", m.group(1))]),
        (r'([0-9]{1,4}동)', lambda m: [("<주소>", m.group(1))]),
        (r'([가-힣A-Za-z0-9]{1,30}(아파트|오피스텔|주택|고시원|빌딩|건물|맨션|연립주택|사옥|하우스|타워|상가|점포|회관|센터|몰|모델하우스|아울렛|백화점))(?=\s|' + JOSA + r'|\b)',
         lambda m: [("<건물>", m.group(1))]),
        (r'\b(건물|아파트|빌딩|주택|하우스)\b', lambda m: [("<건물>", m.group(1))]),
        (r'([가-힣A-Za-z0-9]{1,30}(호텔|모텔|펜션))', lambda m: [("<숙박시설>", m.group(1))]),
        (r'([가-힣]{2,20}공원)', lambda m: [("<공원>", m.group(1))]),
        (r'([가-힣]{2,10}공고)', lambda m: [("<학교>", m.group(1))]),
        (rf'\b(?!안전?사고)([가-힣]{{2,10}}고)(?={JOSA})', lambda m: [("<학교>", m.group(1))]),
        (rf'\b((?!{EXCLUDE_FOR_STATION})[가-힣0-9]{{2,20}}역)(?=\s|' + JOSA + r'|\b)',
         lambda m: [("<역>", m.group(1))]),
        (EXCLUDE_PATTERN, lambda m: [("##EXCLUDE##", m.group(0))]),
    ]

    matches = []
    replaced_spans = []

    # ✅ 원본 text 기준으로 전체 매치 수집
    for pattern, slot_func in patterns:
        for match in re.finditer(pattern, text):
            start, end = match.span()
            if any(s < end and start < e for s, e in replaced_spans):
                continue
            for slot, value in slot_func(match):
                if slot != "##EXCLUDE##":
                    matches.append((start, end, slot, value))
                    replaced_spans.append((start, end))
                    slot_values[slot] = value  # ✅ 항상 마지막 slot_value만 저장됨
                    break

    # ✅ masked_text를 뒤에서 앞으로 치환하여 인덱스 보존
    masked_text = text
    for start, end, slot, _ in sorted(matches, reverse=True):
        masked_text = masked_text[:start] + slot + masked_text[end:]

    return masked_text, slot_values


# ✅ 테스트
text = "금일 10:10 고잔동 102-57번지 삼성화재 광주상무사옥 102-57호에 화재 발생. 이 지역을 우회하여 주시고 인근 주민은 안전사고 발생에 유의 바랍니다."
masked, slots = grouped_region_mask_korean_text_final(text)
print("🔹 Masked Text:", masked)
print("🔑 Slot Values:", slots)

In [ ]:
# ✅ 마스킹 적용 및 출력
TARGET_DIR = "/teamspace/studios/this_studio/training/train/norm_keypoint"
print_limit = 200
masked_results = []
import re

for path in sorted(glob(os.path.join(TARGET_DIR, "*.json")))[:print_limit]:
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    korean_text = data.get("korean_text", "")
    masked_text, slot_values = grouped_region_mask_korean_text_final(korean_text)

    # 저장
    data["masked_korean_text"] = masked_text
    data["slot_values"] = slot_values
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    masked_results.append((os.path.basename(path), korean_text, masked_text, slot_values))

for i, (fname, orig, masked, slots) in enumerate(masked_results[60:80], 1):
    print(f"📁 {i:02d}. File: {fname}")
    print(f"🔸 Original: {orig}")
    print(f"🔹 Masked  : {masked}")
    print(f"🔑 Slot Values: {json.dumps(slots, ensure_ascii=False)}\n{'-'*60}")

In [53]:
from pathlib import Path
import json
from tqdm import tqdm

# 📌 마스킹 함수는 이미 정의되었다고 가정 (grouped_region_mask_korean_text_final)

def apply_masking_to_json_dir(json_dir: str, field="korean_text"):
    for file in tqdm(list(Path(json_dir).glob("*.json")), desc=f"📂 Masking in {json_dir}"):
        with open(file, encoding="utf-8") as f:
            data = json.load(f)

        if field in data:
            text = data[field].strip()
            masked_text, slot_values = grouped_region_mask_korean_text_final(text)
            data["masked_text"] = masked_text
            data["slot_values"] = slot_values

            with open(file, "w", encoding="utf-8") as f:
                json.dump(data, f, ensure_ascii=False, indent=2)

# ✅ 경로 지정
train_dir = Path("training/train/norm_keypoint")
val_dir = Path("training/val/norm_keypoint")

# ✅ 적용
apply_masking_to_json_dir(train_dir)
apply_masking_to_json_dir(val_dir)

📂 Masking in training/val/norm_keypoint: 100%|██████████| 408/408 [00:33<00:00, 12.28it/s]


In [55]:
# Augmentation via Skip Sampling (Masking 이후)

import os
import json
import random
import numpy as np
import copy
from glob import glob
from tqdm import tqdm

# 📁 디렉토리 설정
NORM_DIR = "/teamspace/studios/this_studio/training/train/norm_keypoint"
AUG_DIR = "/teamspace/studios/this_studio/training/train/augmented_keypoint"
os.makedirs(AUG_DIR, exist_ok=True)

# 📌 하이퍼파라미터 설정
AUG_FACTOR = 50
N_FRAMES_SAMPLED = 100

# 파일 목록
file_list = sorted(glob(os.path.join(NORM_DIR, "*_norm.json")))

# 중복 frame 발생 수 기록용
duplicate_aug_count = 0

# 🔧 균일 sampling 인덱스 계산 함수
def safe_sampling_indices(l, n):
    z = int(np.floor(l / (n - 1)))
    y = int(np.floor((l - z * (n - 1)) / 2))
    baseline_idx = [y + z * k for k in range(n)]

    # soft clipping
    if baseline_idx[-1] >= l:
        baseline_idx[-1] = l - 1

    return baseline_idx, z

# 🔁 파일별 augmentation 적용
for src_path in tqdm(file_list, desc="🔁 Skip Sampling Augmentation"):
    basename = os.path.basename(src_path)
    file_idx = basename.replace('_norm.json', '')

    with open(src_path, 'r', encoding='utf-8') as f:
        src_data = json.load(f)

    frames = src_data['frames']
    l = len(frames)
    n = N_FRAMES_SAMPLED

    if l < n:
        # 🔸 길이 부족 시 복제 padding
        indices = list(range(l)) + [l - 1] * (n - l)
        sampled_frames = [frames[i] for i in indices]

        new_data = {
            "id": src_data["id"],
            "masked_text": src_data["masked_text"],
            "frames": sampled_frames,
        }
        for i, frame in enumerate(new_data["frames"]):
            frame["frame_idx"] = i

        save_path = os.path.join(AUG_DIR, f"augmented_{file_idx}_1.json")
        with open(save_path, 'w', encoding='utf-8') as wf:
            json.dump(new_data, wf, ensure_ascii=False, indent=2)

    else:
        baseline_idx, z = safe_sampling_indices(l, n)

        if z == 1:
            # 🔸 간격이 1이면 동일한 결과만 나오므로 1개만 생성
            indices = [min(l - 1, max(0, idx)) for idx in baseline_idx]
            sampled_frames = [frames[i] for i in indices]

            new_data = {
                "id": src_data["id"],
                "masked_text": src_data["masked_text"],
                "frames": sampled_frames,
            }
            for i, frame in enumerate(new_data["frames"]):
                frame["frame_idx"] = i

            save_path = os.path.join(AUG_DIR, f"augmented_{file_idx}_1.json")
            with open(save_path, 'w', encoding='utf-8') as wf:
                json.dump(new_data, wf, ensure_ascii=False, indent=2)

        else:
            # 🔸 다양한 noise 기반 augmentation 생성
            for aug_idx in range(1, AUG_FACTOR + 1):
                indices = []
                for k in range(n):
                    base = baseline_idx[k]
                    if base >= l - 1:
                        noise = 0
                    else:
                        noise_max = max(1, l - 1 - base) if k == n - 1 else max(1, z)
                        noise = random.randint(0, noise_max - 1)
                    idx = min(base + noise, l - 1)
                    indices.append(idx)

                if len(set(indices)) < len(indices):
                    duplicate_aug_count += 1

                sampled_frames = [frames[i] for i in indices]

                new_data = {
                    "id": src_data["id"],
                    "masked_text": src_data["masked_text"],
                    "frames": sampled_frames,
                }
                for i, frame in enumerate(new_data["frames"]):
                    frame["frame_idx"] = i

                save_path = os.path.join(AUG_DIR, f"augmented_{file_idx}_{aug_idx}.json")
                with open(save_path, 'w', encoding='utf-8') as wf:
                    json.dump(new_data, wf, ensure_ascii=False, indent=2)

# ✅ 요약 출력
print(f"\n✅ 모든 train 파일에서 augmentation 결과를 '{AUG_DIR}'에 저장했습니다.")
print(f"🔍 중복된 frame이 포함된 augmentation 수: {duplicate_aug_count:,}")


🔁 Skip Sampling Augmentation: 100%|██████████| 3183/3183 [29:31<00:00,  1.80it/s]


✅ 모든 train 파일에서 augmentation 결과를 '/teamspace/studios/this_studio/training/train/augmented_keypoint'에 저장했습니다.
🔍 중복된 frame이 포함된 augmentation 수: 217


In [59]:
!pip install ujson

In [60]:
# 코드 상태 초기화로 재실행
import os
import ujson  # ultra fast JSON parser
from glob import glob

# 디렉토리 설정
AUG_DIR = "/teamspace/studios/this_studio/training/train/augmented_keypoint"

# 체크용 결과 저장
invalid_frame_counts = []
missing_masked_text = []

# 파일 목록
aug_files = glob(os.path.join(AUG_DIR, "augmented_*.json"))

# 빠른 검사
for file_path in aug_files:
    with open(file_path, "r", encoding="utf-8") as f:
        data = ujson.load(f)

    if len(data.get("frames", [])) != 100:
        invalid_frame_counts.append(os.path.basename(file_path))

    if not data.get("masked_text", "").strip():
        missing_masked_text.append(os.path.basename(file_path))

{
    "총 파일 수": len(aug_files),
    "frame 수가 100이 아닌 파일 수": len(invalid_frame_counts),
    "masked_text가 없는 파일 수": len(missing_masked_text),
    "예시 frame 오류 파일": invalid_frame_counts[:3],
    "예시 masked_text 오류 파일": missing_masked_text[:3]
}


{'총 파일 수': 159150,
 'frame 수가 100이 아닌 파일 수': 0,
 'masked_text가 없는 파일 수': 0,
 '예시 frame 오류 파일': [],
 '예시 masked_text 오류 파일': []}

In [41]:
import os
import json
import numpy as np
from tqdm import tqdm

# ✅ 경로 설정
DATA_DIR = "./training/train/augmented_keypoint"
SAVE_DIR = "./training/train/processed"
PART_SIZE = 10_000
EXPECTED_FRAME_LEN = 100

os.makedirs(SAVE_DIR, exist_ok=True)

# 📄 JSON 파일 목록
file_list = sorted([
    os.path.join(DATA_DIR, f)
    for f in os.listdir(DATA_DIR)
    if f.endswith(".json")
])
total_files = len(file_list)

# 🔁 변환용 리스트 초기화
X_list, text_list, file_id_list = [], [], []
part_idx = 0

progress = tqdm(file_list, desc="🔄 Converting JSON to Numpy")

for i, file_path in enumerate(progress):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    masked_text = data.get("masked_text", "").strip()
    if not masked_text:
        tqdm.write(f"⚠️ Skipped (no masked_text): {file_path}")
        continue

    frames = data.get("frames", [])
    if len(frames) != EXPECTED_FRAME_LEN:
        tqdm.write(f"⚠️ Skipped (wrong frame length = {len(frames)}): {file_path}")
        continue

    keypoints = []
    for frame in frames:
        pose = frame.get("pose", [])
        hand_l = frame.get("hand_left", [])
        hand_r = frame.get("hand_right", [])
        keypoint = pose + hand_l + hand_r
        if len(keypoint) != 108:
            tqdm.write(f"⚠️ Skipped (bad keypoint length): {file_path}")
            keypoints = []
            break
        keypoints.append(keypoint)

    if len(keypoints) != EXPECTED_FRAME_LEN:
        continue

    X_list.append(keypoints)
    text_list.append(masked_text)
    file_id = os.path.basename(file_path).replace(".json", "")
    file_id_list.append(file_id)

    if len(X_list) == PART_SIZE or i == total_files - 1:
        X_array = np.array(X_list, dtype=np.float32)
        np.save(os.path.join(SAVE_DIR, f"X_part_{part_idx}.npy"), X_array)
        np.save(os.path.join(SAVE_DIR, f"file_id_part_{part_idx}.npy"), np.array(file_id_list))

        with open(os.path.join(SAVE_DIR, "spm_input.txt"), "a", encoding="utf-8") as txt_file:
            for line in text_list:
                txt_file.write(line.strip() + "\n")

        tqdm.write(f"✅ Saved X_part_{part_idx}.npy → shape {X_array.shape}")
        tqdm.write(f"📎 file_id_part_{part_idx}.npy → {len(file_id_list)}개")

        X_list, text_list, file_id_list = [], [], []
        part_idx += 1

tqdm.write("\n🎉 All data has been converted and saved.")


🔄 Converting JSON to Numpy:   6%|▋         | 9988/159150 [00:47<10:24, 238.79it/s]

✅ Saved X_part_0.npy → shape (10000, 100, 108)
📎 file_id_part_0.npy → 10000개


🔄 Converting JSON to Numpy:  13%|█▎        | 19980/159150 [01:35<09:24, 246.38it/s] 

✅ Saved X_part_1.npy → shape (10000, 100, 108)
📎 file_id_part_1.npy → 10000개


🔄 Converting JSON to Numpy:  19%|█▉        | 29999/159150 [02:23<08:37, 249.55it/s] 

...

✅ Saved X_part_14.npy → shape (10000, 100, 108)
📎 file_id_part_14.npy → 10000개


🔄 Converting JSON to Numpy: 100%|█████████▉| 159142/159150 [12:44<00:00, 258.54it/s]

✅ Saved X_part_15.npy → shape (9150, 100, 108)
📎 file_id_part_15.npy → 9150개


🔄 Converting JSON to Numpy: 100%|██████████| 159150/159150 [12:45<00:00, 207.97it/s]


🎉 All data has been converted and saved.


In [42]:
import sentencepiece as spm

# 학습용 txt 경로
input_txt = "./training/train/processed/spm_input.txt"

# 저장 prefix
model_prefix = "./training/train/processed/spm"
vocab_size = 1000  # ✅ vocab 사이즈 확장

# SentencePiece 모델 학습
spm.SentencePieceTrainer.Train(
    input=input_txt,
    model_prefix=model_prefix,
    vocab_size=vocab_size,
    character_coverage=0.9995,
    model_type="bpe",
    unk_id=0,              # ✅ UNK = 0 (기본값과 동일하지만 명시적으로 지정)
    bos_id=1,              # ✅ BOS = 1
    eos_id=2,              # ✅ EOS = 2
    pad_id=3,              # ✅ PAD = 3
    pad_piece="<pad>",     # ✅ PAD 토큰 명시
    user_defined_symbols=[
        "<건물>", "<지역>", "<시간>", "<도로>", "<주소>",
        "<학교>", "<산>", "<공원>", "<역>", "<숙박시설>"
    ]
)

print("✅ SentencePiece tokenizer 학습 완료!")

✅ SentencePiece tokenizer 학습 완료!


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: ./training/train/processed/spm_input.txt
  input_format: 
  model_prefix: ./training/train/processed/spm
  model_type: BPE
  vocab_size: 1000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  user_defined_symbols: <건물>
  user_defined_symbols: <지역>
  user_defined_symbols: <시간>
  user_defined_symbols: <도로>
  user_defined_symbols: <주소>
  user_defined_symbols: <학교>
  user_defined_symbols: <산>
  user_defined_symbols: <공원>
  user_defined_symbols: <역>
  user_defined_symbols: <숙박시설>
  required_chars: 
  b

In [43]:
import os
import json
import numpy as np
from tqdm import tqdm
import sentencepiece as spm

# 📌 SentencePiece tokenizer 로드
sp = spm.SentencePieceProcessor()
sp.load("./training/train/processed/spm.model")

bos_id = sp.bos_id()   # 1
eos_id = sp.eos_id()   # 2

# 📁 디렉토리 설정
DATA_DIR = "./training/train/augmented_keypoint"
SAVE_DIR = "./training/train/processed"
PART_SIZE = 10_000

os.makedirs(SAVE_DIR, exist_ok=True)

# 📄 JSON 파일 목록
file_list = sorted([
    os.path.join(DATA_DIR, f)
    for f in os.listdir(DATA_DIR)
    if f.endswith(".json")
])

# 📦 저장 리스트 초기화
y_list = []
part_idx = 0

# 🔁 파일별 처리
for i, file_path in enumerate(tqdm(file_list, desc="🌤️ Encoding masked_text to y.npy")):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    masked_text = data.get("masked_text", "").strip()
    if not masked_text:
        continue

    # SentencePiece로 인코딩 (BOS/EOS 포함)
    y_ids = [bos_id] + sp.encode(masked_text, out_type=int) + [eos_id]
    y_list.append(y_ids)

    # PART 단위 저장
    if len(y_list) == PART_SIZE or i == len(file_list) - 1:
        np.save(os.path.join(SAVE_DIR, f"y_part_{part_idx}.npy"), np.array(y_list, dtype=object))
        print(f"✅ Saved y_part_{part_idx}.npy → {len(y_list)} sequences")
        y_list = []
        part_idx += 1

print("\n🎉 All masked_text → y numpy 변환 완료!")

🌤️ Encoding masked_text to y.npy:   6%|▋         | 10028/159150 [00:37<09:01, 275.35it/s]

✅ Saved y_part_0.npy → 10000 sequences


🌤️ Encoding masked_text to y.npy:  13%|█▎        | 20051/159150 [01:16<09:08, 253.40it/s]

✅ Saved y_part_1.npy → 10000 sequences


🌤️ Encoding masked_text to y.npy:  19%|█▉        | 30049/159150 [01:54<08:43, 246.44it/s]

...

✅ Saved y_part_13.npy → 10000 sequences


🌤️ Encoding masked_text to y.npy:  94%|█████████▍| 150042/159150 [09:30<00:33, 274.61it/s]

✅ Saved y_part_14.npy → 10000 sequences


🌤️ Encoding masked_text to y.npy: 100%|██████████| 159150/159150 [10:05<00:00, 262.96it/s]

✅ Saved y_part_15.npy → 9150 sequences

🎉 All masked_text → y numpy 변환 완료!


In [44]:
# Processing Validation set 

import os
import json
import numpy as np
import sentencepiece as spm
from tqdm import tqdm

# 🔧 경로 설정
VAL_DATA_DIR = "/teamspace/studios/this_studio/training/val/norm_keypoint"
SAVE_DIR = "/teamspace/studios/this_studio/training/val/processed"
SPM_MODEL = "/teamspace/studios/this_studio/training/train/processed/spm.model"
PART_SIZE = 10_000

os.makedirs(SAVE_DIR, exist_ok=True)

# 🔁 SentencePiece tokenizer 로드
sp = spm.SentencePieceProcessor()
sp.load(SPM_MODEL)

bos_id = sp.bos_id()  # 1
eos_id = sp.eos_id()  # 2

# 📄 JSON 파일 목록 정렬
file_list = sorted([
    os.path.join(VAL_DATA_DIR, f)
    for f in os.listdir(VAL_DATA_DIR)
    if f.endswith(".json")
])
total_files = len(file_list)

# 📦 저장용 리스트 초기화
x_list, y_list, file_id_list = [], [], []
part_idx = 0

for i, file_path in enumerate(tqdm(file_list, desc="🔄 Processing Validation Set")):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # 1️⃣ masked_text 추출
    masked_text = data.get("masked_text", "").strip()
    if not masked_text:
        continue

    # 2️⃣ keypoint 추출
    frames = data.get("frames", [])
    if len(frames) < 1:
        continue

    keypoints = []
    for frame in frames:
        pose = frame.get("pose", [])
        hand_l = frame.get("hand_left", [])
        hand_r = frame.get("hand_right", [])
        keypoint = pose + hand_l + hand_r
        if len(keypoint) != 108:
            break
        keypoints.append(keypoint)

    if len(keypoints) < 1:
        continue

    # 3️⃣ 변환 및 저장
    x_list.append(np.array(keypoints, dtype=np.float32))  # variable-length
    y_ids = [bos_id] + sp.encode(masked_text, out_type=int) + [eos_id]
    y_list.append(y_ids)
    file_id = os.path.basename(file_path).replace(".json", "")
    file_id_list.append(file_id)

    # 🔽 저장 조건
    if len(x_list) == PART_SIZE or i == total_files - 1:
        np.save(os.path.join(SAVE_DIR, f"x_part_{part_idx}.npy"), np.array(x_list, dtype=object))
        np.save(os.path.join(SAVE_DIR, f"y_part_{part_idx}.npy"), np.array(y_list, dtype=object))
        np.save(os.path.join(SAVE_DIR, f"file_id_part_{part_idx}.npy"), np.array(file_id_list))

        print(f"✅ Saved x_part_{part_idx}.npy → {len(x_list)} samples")
        print(f"✅ Saved y_part_{part_idx}.npy → {len(y_list)} samples")

        # 초기화
        x_list, y_list, file_id_list = [], [], []
        part_idx += 1

print("\n🎉 Validation set 변환 및 저장 완료 (masked_text 기준, variable-length 허용)")

🔄 Processing Validation Set: 100%|██████████| 408/408 [00:09<00:00, 40.83it/s]

✅ Saved x_part_0.npy → 408 samples
✅ Saved y_part_0.npy → 408 samples

🎉 Validation set 변환 및 저장 완료 (masked_text 기준, variable-length 허용)


In [25]:
import os
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

PAD_ID = 3  # ✅ 패딩 토큰은 항상 3으로 고정

# -----------------------------
# 1. SLTDataset (Train Dataset)
# -----------------------------
class SLTDataset(Dataset):
    def __init__(self, x_dir, y_dir, part_ids):
        self.x_paths = [os.path.join(x_dir, f"X_part_{i}.npy") for i in part_ids]
        self.y_paths = [os.path.join(y_dir, f"y_part_{i}.npy") for i in part_ids]  # ✅ 소문자 y로 변경

        self.X = []
        self.y = []

        for x_path, y_path in zip(self.x_paths, self.y_paths):
            self.X.extend(np.load(x_path, allow_pickle=True))
            self.y.extend(np.load(y_path, allow_pickle=True))

        assert len(self.X) == len(self.y), "X and y size mismatch"

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = torch.tensor(self.X[idx], dtype=torch.float32)  # (T, 108)
        y = torch.tensor(self.y[idx], dtype=torch.long)     # (L,)
        return x, y


# -----------------------------
# 2. Collate Function
# -----------------------------
def collate_fn(batch, pad_id=PAD_ID, max_len=100):
    xs, ys = zip(*batch)

    # 🧩 x padding
    max_x_len = min(max([x.shape[0] for x in xs]), max_len)
    padded_xs = []
    for x in xs:
        if x.shape[0] >= max_x_len:
            padded_xs.append(x[:max_x_len])
        else:
            pad = torch.zeros(max_x_len - x.shape[0], x.shape[1])
            padded_xs.append(torch.cat([x, pad], dim=0))
    xs = torch.stack(padded_xs)  # (B, T, 108)

    # 🧩 y padding
    max_y_len = max([len(y) for y in ys])
    ys = [F.pad(y, (0, max_y_len - len(y)), value=pad_id) for y in ys]
    ys = torch.stack(ys)

    return xs, ys


# -----------------------------
# 3. Train DataLoader
# -----------------------------
X_DIR = "/teamspace/studios/this_studio/training/train/processed"
Y_DIR = X_DIR
PART_IDS = list(range(16))

train_dataset = SLTDataset(X_DIR, Y_DIR, PART_IDS)
train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=2
)


# -----------------------------
# 4. SLTValDataset (Validation)
# -----------------------------
class SLTValDataset(Dataset):
    def __init__(self, data_dir):
        self.X_paths = sorted([os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.startswith("X_part_")])
        self.y_paths = sorted([os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.startswith("y_part_")])  # ✅ 소문자 y

        self.X = []
        self.y = []

        for x_path, y_path in zip(self.X_paths, self.y_paths):
            self.X.extend(np.load(x_path, allow_pickle=True))
            self.y.extend(np.load(y_path, allow_pickle=True))

        assert len(self.X) == len(self.y), "X and y size mismatch in validation set"

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = torch.tensor(self.X[idx], dtype=torch.float32)  # (T, 108)
        y = torch.tensor(self.y[idx], dtype=torch.long)     # (L,)
        return x, y


def get_validation_loader(val_dir, batch_size=64, num_workers=2):
    val_dataset = SLTValDataset(val_dir)
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        collate_fn=collate_fn
    )
    return val_loader


In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# -------------------------
# 1. Attention 모듈
# -------------------------
class Attention(nn.Module):
    def __init__(self, encoder_hidden_dim, decoder_hidden_dim):
        super().__init__()
        self.attn = nn.Linear(encoder_hidden_dim + decoder_hidden_dim, decoder_hidden_dim)
        self.v = nn.Linear(decoder_hidden_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        # hidden: (B, dec_hidden)
        # encoder_outputs: (B, src_len, enc_hidden_dim)
        src_len = encoder_outputs.size(1)
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)  # (B, src_len, dec_hidden)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))  # (B, src_len, dec_hidden)
        attention = self.v(energy).squeeze(2)  # (B, src_len)
        return F.softmax(attention, dim=1)


# -------------------------
# 2. Encoder (BiGRU)
# -------------------------
class Encoder(nn.Module):
    def __init__(self, input_dim=108, hidden_dim=256):
        super().__init__()
        self.rnn = nn.GRU(input_dim, hidden_dim, batch_first=True, bidirectional=True)

    def forward(self, src):
        # src: (B, T, input_dim)
        outputs, hidden = self.rnn(src)  # outputs: (B, T, 2*H), hidden: (2, B, H)
        return outputs, hidden


# -------------------------
# 3. Decoder (GRU + Attention)
# -------------------------
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim=256, enc_hidden_dim=256, dec_hidden_dim=256):
        super().__init__()
        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.attention = Attention(enc_hidden_dim * 2, dec_hidden_dim)
        self.rnn = nn.GRU(emb_dim + enc_hidden_dim * 2, dec_hidden_dim, batch_first=True)
        self.fc_out = nn.Linear(emb_dim + enc_hidden_dim * 2 + dec_hidden_dim, output_dim)

    def forward(self, input, hidden, encoder_outputs):
        # input: (B,)
        # hidden: (1, B, dec_hidden)
        # encoder_outputs: (B, src_len, enc_hidden*2)
        input = input.unsqueeze(1)  # (B, 1)
        embedded = self.embedding(input)  # (B, 1, emb_dim)

        attn_weights = self.attention(hidden.squeeze(0), encoder_outputs)  # (B, src_len)
        attn_weights = attn_weights.unsqueeze(1)  # (B, 1, src_len)
        context = torch.bmm(attn_weights, encoder_outputs)  # (B, 1, enc_hidden*2)

        rnn_input = torch.cat((embedded, context), dim=2)  # (B, 1, emb_dim + ctx)
        output, hidden = self.rnn(rnn_input, hidden)  # output: (B, 1, dec_hidden)

        concat = torch.cat((output, context, embedded), dim=2)  # (B, 1, total_dim)
        prediction = self.fc_out(concat.squeeze(1))  # (B, output_dim)

        return prediction, hidden, attn_weights.squeeze(1)


# -------------------------
# 4. Seq2Seq (통합 모듈)
# -------------------------
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, pad_id=3):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.pad_id = pad_id

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: (B, T, 108), trg: (B, L)
        batch_size = src.size(0)
        trg_len = trg.size(1)
        output_dim = self.decoder.output_dim

        # 🧱 Output tensor 초기화
        outputs = torch.zeros(batch_size, trg_len, output_dim).to(self.device)

        # 🔁 Encoder
        encoder_outputs, hidden = self.encoder(src)  # hidden: (2, B, H)
        hidden = torch.tanh(hidden[0] + hidden[1]).unsqueeze(0)  # (1, B, H)

        # 🔁 Decoder 시작 - 첫 input은 BOS 토큰
        input = trg[:, 0]  # (B,)

        for t in range(1, trg_len):
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs)  # output: (B, vocab)
            outputs[:, t] = output  # t 위치에 output 저장

            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(1)  # (B,)

            input = trg[:, t] if teacher_force else top1

        return outputs  # (B, L, vocab)

In [4]:
!pip install evaluate

In [47]:
# Model_1

import os
import torch
import torch.nn.functional as F
from tqdm import tqdm
import evaluate

# ---------------------------
# 1. 평가 지표 로드
# ---------------------------
bleu = evaluate.load("bleu")
meteor = evaluate.load("meteor")
rouge = evaluate.load("rouge")

# ---------------------------
# 2. 디코딩 함수
# ---------------------------
def decode_sequence(tokenizer, sequences, eos_id=2):
    decoded = []
    for seq in sequences:
        tokens = []
        for tok in seq:
            if tok == eos_id:
                break
            tokens.append(tok)
        sentence = tokenizer.decode(tokens)
        decoded.append(sentence.replace("▁", " ").strip())
    return decoded

# ---------------------------
# 3. 평가 지표 계산
# ---------------------------
def compute_metrics(preds, refs, tokenizer, eos_id=2):
    decoded_preds = decode_sequence(tokenizer, preds, eos_id)
    decoded_refs = decode_sequence(tokenizer, refs, eos_id)
    return {
        "BLEU": bleu.compute(predictions=decoded_preds, references=[[r] for r in decoded_refs])["bleu"],
        "METEOR": meteor.compute(predictions=decoded_preds, references=decoded_refs)["meteor"],
        "ROUGE": rouge.compute(predictions=decoded_preds, references=decoded_refs)["rougeL"]
    }

# ---------------------------
# 4. 체크포인트 저장
# ---------------------------
def save_checkpoint(model, optimizer, epoch, val_bleu, val_meteor, val_loss, path):
    checkpoint = {
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "epoch": epoch,
        "val_bleu": val_bleu,
        "val_meteor": val_meteor,
        "val_loss": val_loss
    }
    torch.save(checkpoint, path)


# ---------------------------
# 5. 학습 루프 with EOS/BOS + 샘플 출력
# ---------------------------
def train_with_early_stopping(
    model, train_loader, val_loader, tokenizer,
    optimizer, criterion, device,
    num_epochs=30,
    teacher_forcing_ratio=0.5,
    clip=1.0,
    pad_id=0,
    eos_id=2,
    save_path="best_model.pt",
    early_stopping_patience=5,
    checkpoint_dir="checkpoints"
):
    os.makedirs(checkpoint_dir, exist_ok=True)
    best_bleu = -1
    patience = 0

    for epoch in range(1, num_epochs + 1):
        model.train()
        epoch_loss = 0
        pbar = tqdm(train_loader, desc=f"[Epoch {epoch}] Training")

        for X, y in pbar:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            decoder_input = y[:, :-1]
            target = y[:, 1:]

            output = model(X, decoder_input, teacher_forcing_ratio=teacher_forcing_ratio)
            output = output.reshape(-1, output.shape[-1])
            target = target.reshape(-1)

            loss = criterion(output, target)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()

            epoch_loss += loss.item()
            pbar.set_postfix(train_loss=epoch_loss / (pbar.n + 1))

        # -----------------------
        # Train Sample 출력
        # -----------------------
        model.eval()
        train_preds, train_refs = [], []
        with torch.no_grad():
            for X_train, y_train in train_loader:
                X_train, y_train = X_train.to(device), y_train.to(device)
                output = model(X_train, y_train[:, :-1], teacher_forcing_ratio=0.0)
                output_tokens = output.argmax(-1)
                train_preds = decode_sequence(tokenizer, output_tokens[:5].tolist(), eos_id)
                train_refs = decode_sequence(tokenizer, y_train[:5].tolist(), eos_id)
                break  # 첫 batch만

        print("\n🟦 [Train Samples]")
        for i in range(len(train_preds)):
            print(f"🔹 [Pred] {train_preds[i]}")
            print(f"🔸 [True] {train_refs[i]}")
            print()

        # -----------------------
        # Validation
        # -----------------------
        val_loss = 0
        preds, refs = [], []

        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                decoder_input = y_val[:, :-1]
                target = y_val[:, 1:]

                output = model(X_val, decoder_input, teacher_forcing_ratio=0.0)
                output_tokens = output.argmax(-1)

                preds.extend(output_tokens.tolist())
                refs.extend(y_val.tolist())

                output = output.reshape(-1, output.shape[-1])
                target = target.reshape(-1)
                val_loss += criterion(output, target).item()

        val_loss /= len(val_loader)
        metrics = compute_metrics(preds, refs, tokenizer, eos_id)
        val_bleu = metrics["BLEU"]
        val_preds = decode_sequence(tokenizer, preds[:5], eos_id)
        val_refs = decode_sequence(tokenizer, refs[:5], eos_id)

        print("\n🟩 [Validation Samples]")
        for i in range(len(val_preds)):
            print(f"🔹 [Pred] {val_preds[i]}")
            print(f"🔸 [True] {val_refs[i]}")
            print()

        print(f"[Epoch {epoch}] Val Loss: {val_loss:.4f} | BLEU: {val_bleu:.4f} | METEOR: {metrics['METEOR']:.4f} | ROUGE: {metrics['ROUGE']:.4f}")
        # ⬅️ 1. 현재 epoch checkpoint 저장 (일반)
        ckpt_path = os.path.join(checkpoint_dir, f"checkpoint_epoch{epoch}.pt")
        save_checkpoint(
            model,
            optimizer,
            epoch,
            val_bleu,
            metrics["METEOR"],
            val_loss,
            ckpt_path  # ✅ 여기에 저장!
        )
        
        # ⬅️ 2. BLEU 기준 best model 저장
        if val_bleu > best_bleu:
            best_bleu = val_bleu
            patience = 0
            torch.save(model.state_dict(), save_path)
            save_checkpoint(
                model,
                optimizer,
                epoch,
                val_bleu,
                metrics["METEOR"],
                val_loss,
                os.path.join(checkpoint_dir, "best_checkpoint.pt")  # ✅ 여기에만 best 저장
            )
            print(f"✅ Best model saved at epoch {epoch} (BLEU={val_bleu:.4f})")
        else:
            patience += 1
            if patience >= early_stopping_patience:
                print(f"🛑 Early stopping triggered. Best BLEU: {best_bleu:.4f}")
                break

[nltk_data] Downloading package wordnet to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [27]:
# Model_2

import os
import torch
import torch.nn.functional as F
from tqdm import tqdm
import evaluate

# ---------------------------
# 1. 평가 지표 로드 (HuggingFace Evaluate)
# ---------------------------
bleu = evaluate.load("bleu")
meteor = evaluate.load("meteor")
rouge = evaluate.load("rouge")


# ---------------------------
# 2. 토큰 시퀀스를 자연어로 복원
# (배치 디코딩용, eos_id 기준으로 잘라냄)
# ---------------------------

import torch
import torch.nn.functional as F

def top_k_top_p_filtering(logits, top_k=0, top_p=1.0, filter_value=-float("Inf")):
    """
    Filter a distribution of logits using top-k and/or nucleus (top-p) filtering.
    """
    logits = logits.clone()

    # Top-k filtering
    if top_k > 0:
        top_k = min(top_k, logits.size(-1))  # Safety check
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    # Top-p (nucleus) filtering
    if top_p < 1.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative prob > top_p
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift right to keep at least one token
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value

    return logits

def decode_sequence(
    model,
    input_tensor,
    tokenizer,
    device,
    max_len=100,
    eos_id=2,
    repetition_penalty=1.2,
    top_k=5,
    top_p=0.9
):
    """
    Sampling-based decoding using top-k, top-p, and repetition penalty.
    """
    model.eval()
    input_tensor = input_tensor.unsqueeze(0).to(device)  # (1, T, D)

    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(input_tensor)
        hidden = torch.tanh(hidden[0] + hidden[1]).unsqueeze(0)  # (1, 1, H)

        input_token = torch.tensor([tokenizer.bos_id()], device=device)  # (1,)
        decoded = []
        used_tokens = set()

        for _ in range(max_len):
            output, hidden, _ = model.decoder(input_token, hidden, encoder_outputs)  # output: (1, vocab)
            output = output.squeeze(0)  # (vocab,)

            # Apply repetition penalty
            for token_id in used_tokens:
                if output[token_id] < 0:
                    output[token_id] *= repetition_penalty
                else:
                    output[token_id] /= repetition_penalty

            # Sampling with top-k and/or top-p
            filtered_logits = top_k_top_p_filtering(output, top_k=top_k, top_p=top_p)
            probs = F.softmax(filtered_logits, dim=-1)
            top1 = torch.multinomial(probs, num_samples=1).item()

            if top1 == eos_id:
                break

            decoded.append(top1)
            used_tokens.add(top1)
            input_token = torch.tensor([top1], device=device)

    return tokenizer.decode(decoded).replace("▁", " ").strip()


def decode_tokens(tokenizer, sequences, eos_id=2, pad_id=3):
    """
    sequences: List of token ids (List[List[int]])
    Returns: List of decoded strings
    """
    results = []
    for seq in sequences:
        tokens = []
        for t in seq:
            if t == eos_id or t == pad_id:
                break
            tokens.append(t)
        results.append(tokenizer.decode(tokens).replace("▁", " ").strip())
    return results

# ---------------------------
# 3. BLEU, METEOR, ROUGE 계산
# ---------------------------
def compute_metrics(preds, refs, tokenizer, eos_id=2, pad_id=3):
    decoded_preds = decode_tokens(tokenizer, preds, eos_id, pad_id)
    decoded_refs = decode_tokens(tokenizer, refs, eos_id, pad_id)

    return {
        "BLEU": bleu.compute(predictions=decoded_preds, references=[[r] for r in decoded_refs])["bleu"],
        "METEOR": meteor.compute(predictions=decoded_preds, references=decoded_refs)["meteor"],
        "ROUGE": rouge.compute(predictions=decoded_preds, references=decoded_refs)["rougeL"]
    }

# ---------------------------
# 4. 체크포인트 저장
# ---------------------------
def save_checkpoint(model, optimizer, epoch, val_bleu, val_meteor, val_loss, path):
    checkpoint = {
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "epoch": epoch,
        "val_bleu": val_bleu,
        "val_meteor": val_meteor,
        "val_loss": val_loss
    }
    torch.save(checkpoint, path)


# ---------------------------
# 4. label smoothing 적용
# ---------------------------

import torch.nn as nn
import torch.nn.functional as F

class LabelSmoothingLoss(nn.Module):
    def __init__(self, label_smoothing, tgt_vocab_size, ignore_index=-100):
        super().__init__()
        self.ignore_index = ignore_index
        self.confidence = 1.0 - label_smoothing
        self.smoothing = label_smoothing
        self.vocab_size = tgt_vocab_size

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=-1)  # (B*T, V)
        true_dist = torch.zeros_like(pred)
        true_dist.fill_(self.smoothing / (self.vocab_size - 2))
        ignore = target == self.ignore_index
        target = target.masked_fill(ignore, 0)
        true_dist.scatter_(1, target.unsqueeze(1), self.confidence)
        true_dist.masked_fill_(ignore.unsqueeze(1), 0.0)
        return F.kl_div(pred, true_dist, reduction='batchmean')




def train_with_early_stopping(
    model, train_loader, val_loader, tokenizer,
    optimizer, criterion, device,
    num_epochs=30,
    teacher_forcing_ratio=0.5,
    clip=1.0,
    pad_id=0,
    eos_id=2,
    save_path="best_model.pt",
    early_stopping_patience=5,
    checkpoint_dir="checkpoints"
):
    os.makedirs(checkpoint_dir, exist_ok=True)
    best_bleu = -1
    patience = 0

    for epoch in range(1, num_epochs + 1):
        model.train()
        epoch_loss = 0
        pbar = tqdm(train_loader, desc=f"[Epoch {epoch}] Training")

        for X, y in pbar:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            decoder_input = y[:, :-1]
            target = y[:, 1:]

            output = model(X, decoder_input, teacher_forcing_ratio=teacher_forcing_ratio)
            output = output.reshape(-1, output.shape[-1])
            target = target.reshape(-1)

            loss = criterion(output, target)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()

            epoch_loss += loss.item()
            pbar.set_postfix(train_loss=epoch_loss / (pbar.n + 1))

        # 🔵 Train 샘플 5개 출력
        model.eval()
        train_preds, train_refs = [], []
        with torch.no_grad():
            for X_train, y_train in train_loader:
                X_train, y_train = X_train.to(device), y_train.to(device)
                out = model(X_train, y_train[:, :-1], teacher_forcing_ratio=0.0)
                out_tokens = out.argmax(-1)
                train_preds = [decode_sequence(model, x, tokenizer, device, eos_id=eos_id) for x in X_train[:5]]
                train_refs = [
                    tokenizer.decode([t for t in sent if t != PAD_ID and t != eos_id]).replace("▁", " ").strip()
                    for sent in y_train[:5].tolist()
                ]
                break

        print("\n🟦 [Train Samples]")
        for i in range(len(train_preds)):
            print(f"🔹 [Pred] {train_preds[i]}")
            print(f"🔸 [True] {train_refs[i]}")
            print()

        # 🟢 Validation
        val_loss = 0
        preds, refs = [], []
        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                out = model(X_val, y_val[:, :-1], teacher_forcing_ratio=0.0)
                out_tokens = out.argmax(-1)
                preds.extend(out_tokens.tolist())
                refs.extend(y_val.tolist())

                out = out.reshape(-1, out.shape[-1])
                target = y_val[:, 1:].reshape(-1)
                val_loss += criterion(out, target).item()

        val_loss /= len(val_loader)
        metrics = compute_metrics(preds, refs, tokenizer, eos_id)
        val_bleu = metrics["BLEU"]
        val_preds = [decode_sequence(model, x, tokenizer, device, eos_id=eos_id) for x in X_val[:5]]
        val_refs = decode_tokens(tokenizer, refs[:5], eos_id=eos_id, pad_id=pad_id)

        print("\n🟩 [Validation Samples]")
        for i in range(len(val_preds)):
            print(f"🔹 [Pred] {val_preds[i]}")
            print(f"🔸 [True] {val_refs[i]}")
            print()

        print(f"[Epoch {epoch}] Val Loss: {val_loss:.4f} | BLEU: {val_bleu:.4f} | METEOR: {metrics['METEOR']:.4f} | ROUGE: {metrics['ROUGE']:.4f}")

        # ✅ Checkpoint 저장
        ckpt_path = os.path.join(checkpoint_dir, f"checkpoint_epoch{epoch}.pt")
        save_checkpoint(
            model,
            optimizer,
            epoch,
            val_bleu,
            metrics["METEOR"],
            val_loss,
            ckpt_path  # ✅ 여기에 저장!
        )
        
        # ⬅️ 2. BLEU 기준 best model 저장
        if val_bleu > best_bleu:
            best_bleu = val_bleu
            patience = 0
            torch.save(model.state_dict(), save_path)
            save_checkpoint(
                model,
                optimizer,
                epoch,
                val_bleu,
                metrics["METEOR"],
                val_loss,
                os.path.join(checkpoint_dir, "best_checkpoint.pt")  # ✅ 여기에만 best 저장
            )
            print(f"✅ Best model saved at epoch {epoch} (BLEU={val_bleu:.4f})")
        else:
            patience += 1
            if patience >= early_stopping_patience:
                print(f"🛑 Early stopping triggered. Best BLEU: {best_bleu:.4f}")
                break

[nltk_data] Downloading package wordnet to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [28]:
vocab_size = 1000  # ✅ vocab 사이즈 확장

import torch

# -------------------------
# 0. 디바이스 설정
# -------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    print("✅ Using device:", torch.cuda.get_device_name(0))
else:
    print("⚠️ Using CPU")

# -------------------------
# 1. 하이퍼파라미터 정의
# -------------------------
INPUT_DIM = 108
ENC_HIDDEN_DIM = 256
DEC_HIDDEN_DIM = 256
EMB_DIM = 256
VOCAB_SIZE = vocab_size      # ✅ 최신 SentencePiece vocab size 반영

PAD_ID = 3              # ✅ pad_id 설정 (unk=0, bos=1, eos=2, pad=3)

# -------------------------
# 2. 모델 정의
# -------------------------
encoder = Encoder(input_dim=INPUT_DIM, hidden_dim=ENC_HIDDEN_DIM)
decoder = Decoder(
    output_dim=VOCAB_SIZE,
    emb_dim=EMB_DIM,
    enc_hidden_dim=ENC_HIDDEN_DIM,
    dec_hidden_dim=DEC_HIDDEN_DIM
)

model = Seq2Seq(encoder, decoder, device).to(device)
print("✅ Seq2Seq model initialized and moved to", device)


⚠️ Using CPU
✅ Seq2Seq model initialized and moved to cpu


In [29]:
# 토크나이저 불러오기
import sentencepiece as spm
from tqdm import tqdm  # ✅ 추천 방식
import torch.nn as nn  # 필요시 상단에 추가

spm_tokenizer = spm.SentencePieceProcessor()
spm_tokenizer.load("/teamspace/studios/this_studio/training/train/processed/spm.model")

True

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    print("✅ Using device:", torch.cuda.get_device_name(0))
else:
    print("⚠️ Using CPU")

PAD_ID = 3  # ✅ 반드시 vocab과 일치해야 함

train_with_early_stopping(
    model=model.to(device),
    train_loader=train_loader,
    val_loader=get_validation_loader("/teamspace/studios/this_studio/training/val/processed"),
    tokenizer=spm_tokenizer,
    optimizer=torch.optim.Adam(model.parameters(), lr=1e-3),
    criterion = LabelSmoothingLoss(label_smoothing=0.1, tgt_vocab_size=spm_tokenizer.vocab_size(), ignore_index=PAD_ID),
    # criterion = nn.CrossEntropyLoss(ignore_index=PAD_ID),
    device=device,
    num_epochs=30,
    teacher_forcing_ratio=0.5,
    clip=1.0,
    save_path="best_model.pt",
    early_stopping_patience=5,
    checkpoint_dir="checkpoints"
)

✅ Using device: NVIDIA L40S


[Epoch 1] Training: 100%|██████████| 2487/2487 [02:27<00:00, 16.84it/s, train_loss=0.836]



🟦 [Train Samples]
🔹 [Pred] <시간> <지역> <건물> 화재 발생, 인근주민은 안전한 대피 바람.
🔸 [True] 오늘 <시간> <지역> <건물> 화재 발생, 인근주민은 안전한 곳으로 대피 바람.

🔹 [Pred] <시간> <지역> <지역> 소재 오피스텔 화재발생으로 주변은 교통 우회하시기 바랍니다.
🔸 [True] 오늘 <시간> <지역> <지역> 소재 오피스텔 화재발생으로 주변 교통은 혼잡하오니 우회하시기 바랍니다.

🔹 [Pred] <시간> <지역> <건물> 롯데리아  화재 발생. 주민은 인근 안전한 곳으로 대피하시기 바랍니다.
🔸 [True] 오늘 <시간> <지역> <건물> 롯데리아 <건물> 화재 발생. 인근 주민은 안전한 곳으로 대피하시기 바랍니다.

🔹 [Pred] <시간>경 <지역> <주소> 삼성화재 <건물> 화재로 연기 발생. 주민은 안전사고 안전에 유의 바랍니다
🔸 [True] 오늘 <시간>경 <지역> <주소> 삼성화재 <건물> 화재로 연기 발생. 인근 주민은 안전에 유의 바랍니다.

🔹 [Pred] <시간> <도로>가 화재로 인해 통제중이오니 인근 차량 주민은 주시기 주시기 바랍니다.
🔸 [True] 오늘 <시간> <도로>가 화재로 인해 차량 통제중이오니 인근 차량은 우회하여 주시기 바랍니다.


🟩 [Validation Samples]
🔹 [Pred] <시간> <역>에 화재, 발생, 주민은 닫아주시고 닫아주시고 안전사고에 유의 바랍니다
🔸 [True] 5.23. <시간> <지역> <건물> <건물> 지하주차장 화재로 인해 차량통제 중임을 알려드립니다.

🔹 [Pred] <시간> <지역> <주소> <건물> 화재 발생.. 기기 바랍니다 바랍니다 우회하여 바랍니다.
🔸 [True] 5.23. <시간> <지역> <건물> <건물> 지하주차장 화재로 인해 차량통제 중임을 알려드립니다.

🔹 [Pred] <시간> <역> 인근 <건물> 화재 발생, 인근 주민은 닫아주시고 안전사고에 유의 바랍니다
🔸 [True] 5.23. <시간> <지역

[Epoch 2] Training: 100%|██████████| 2487/2487 [02:18<00:00, 18.02it/s, train_loss=0.342]



🟦 [Train Samples]
🔹 [Pred] <시간> <지역> <주소> SM 오피스텔 화재 발생. 인근 안전에 바랍니다
🔸 [True] 오늘 <시간> <지역> <주소> SM 오피스텔 <건물> 화재 발생. 안전에 유의 바랍니다

🔹 [Pred] 현재 <산> 일대 대형화재발생, <도로>를 시민들께서는 를 바랍니다.
🔸 [True] 오늘 현재 <산> 일대 대형화재발생, <산> 인근 시민들께서는 <도로>를 이용하시기 바랍니다.

🔹 [Pred] <시간>경 <지역> <산>가산길 <주소> 화재발생, 인근 주민은 안전에 유의하기 바랍니다.
🔸 [True] 오늘 <시간>경 <지역> <산>가산길 <주소> 화재발생, 인근 주민은 안전에 유의하기 바랍니다.

🔹 [Pred] <시간> <지역> <지역> <건물> 상가 화재 발생. 인근 주민은 안전사고 발생에 유의 바랍니다
🔸 [True] 오늘 <시간> <지역> <지역> <건물> 상가 화재 발생. 인근 주민은 안전사고 발생에 유의 바랍니다.

🔹 [Pred] <시간> <지역> <주소> 화재 발생. 인근 주민은 안전에 유의 바랍니다.
🔸 [True] 오늘 <시간> <지역> <주소> 화재 발생. 인근 주민은 안전사고에 유의 바랍니다.


🟩 [Validation Samples]
🔹 [Pred] <시간> <역> 화재 발생, 인근 주민은 창문을 닫아주시고 안전사고에 유의 바랍니다.
🔸 [True] 5.23. <시간> <지역> <건물> <건물> 지하주차장 화재로 인해 차량통제 중임을 알려드립니다.

🔹 [Pred] <시간> <지역> <지역> <건물> 인근에서 발생..  주민들은 발생에 유의 바랍니다.
🔸 [True] 5.23. <시간> <지역> <건물> <건물> 지하주차장 화재로 인해 차량통제 중임을 알려드립니다.

🔹 [Pred] <시간> <역> 인근 <건물> 화재발생, 인근 주민은 닫아주시고 안전에 바랍니다.
🔸 [True] 5.23. <시간> <지역> <건물> <건물> 지하주차장 화재로 인해 차량통제 중임을 알려드립니다.

🔹 [P

...


🟦 [Train Samples]
🔹 [Pred] <시간> <지역> <주소> <건물> 화재 발생. 인근 주민은 안전사고에 유의 바랍니다.
🔸 [True] 오늘 <시간> <지역> <주소> <건물> 화재 발생. 인근 주민은 안전사고에 유의 바랍니다.

🔹 [Pred] <시간> <주소>역 1번출구 인근 전주폐차장에서 화재 발생으로 주변 교통이 혼잡하오니 퇴근길 안전에  ⁇ 히 유의 바랍니다
🔸 [True] 오늘 <시간> <주소>역 1번출구 인근 전주폐차장에서 화재 발생으로 주변 교통이 혼잡하오니 퇴근길 안전에  ⁇ 히 유의 바랍니다

🔹 [Pred] <시간> <지역> <지역> <건물> 상가 화재 발생. 인근 주민은 안전사고 발생에 유의 바랍니다.
🔸 [True] 오늘 <시간> <지역> <지역> <건물> 상가 화재 발생. 인근 주민은 안전사고 발생에 유의 바랍니다.

🔹 [Pred] <시간> <지역> <지역> 모토고개 <주소> 재활용공장 화재 발생으로 인근 주민들은 안전사고 발생에 유의 바랍니다.
🔸 [True] 오늘 <시간> <지역> <지역> 모토고개 <주소> 재활용공장 화재 발생으로 인근 주민들은 안전사고 발생에 유의 바랍니다.

🔹 [Pred] <시간> <지역> 봉강저수지 <도로> 화재 발생. 인근 주민은 안전사고에 유의 바랍니다
🔸 [True] <시간> <지역> 봉강저수지 <도로> 화재 발생. 인근 주민은 안전사고에 유의 바랍니다


🟩 [Validation Samples]
🔹 [Pred] <시간> <역> 화재 발생, 인근 주민은 안전에 발생에 유의 바랍니다
🔸 [True] 5.23. <시간> <지역> <건물> <건물> 지하주차장 화재로 인해 차량통제 중임을 알려드립니다.

🔹 [Pred] <시간> <지역> <지역> <건물> 화재 발생. 이<도로> 이용하여 주시기 바랍니다.
🔸 [True] 5.23. <시간> <지역> <건물> <건물> 지하주차장 화재로 인해 차량통제 중임을 알려드립니다.

🔹 [Pred] <시간> 발생한<역>K통신구 화재 잔화정리로, 인근 주

[Epoch 19] Training: 100%|██████████| 2487/2487 [02:15<00:00, 18.35it/s, train_loss=0.25] 



🟦 [Train Samples]
🔹 [Pred] <시간> 구 화장터 화재로 인해 범어로 일대 교통이 정체되고 있으니 우회하여 주시기 바랍니다
🔸 [True] 오늘 <시간> 구 화장터 화재로 인해 범어로 일대 교통이 정체되고 있으니 우회하여 주시기 바랍니다.

🔹 [Pred] <시간> 발생한 <지역> 공장화재 사고 수습으로 수습으로<도로>혼잡하 안전사고에 바랍니다 유의
🔸 [True] 오늘 <시간> 발생한 <지역> <지역> 공장화재 사고 수습으로 <도로>혼잡하니 안전사고에 유의 바랍니다

🔹 [Pred] <시간>현재 <역> 인근 <건물> 화재 발생, 인근 주민은 안전한 곳으로 대피하시기 바랍니다.
🔸 [True] <시간>현재 <역> 인근 <건물> 화재 발생, 인근 주민은 안전한 곳으로 대피하시기 바랍니다.

🔹 [Pred] <시간> 동광교 공사장 공사장 화재로 발생. 인근 주민들은 안전에 유의 바랍니다
🔸 [True] 오늘<시간> 동광교 인근 공사장 화재로 다량의 연기가 발생. 인근 주민은 안전에 유의 바랍니다.

🔹 [Pred] <지역> 대화공단내 대형화재 발생, 출근길 교통혼잡이 예상되오니 대중교통을 이용하시기 바랍니다.
🔸 [True] 현재 <지역> 대화공단내 대형화재 발생, 출근길 교통혼잡이 예상되오니 대중교통을 이용하시기 바랍니다.


🟩 [Validation Samples]
🔹 [Pred] <시간> <역>에 화재, 인근 주민은 창문을 닫아주시고 안전에 유의 바랍니다.
🔸 [True] 5.23. <시간> <지역> <건물> <건물> 지하주차장 화재로 인해 차량통제 중임을 알려드립니다.

🔹 [Pred] <시간> <지역> <지역> 화재 발생. 이 지역을 우회하여 주시고 롯데리아 주민은 유의 바랍니다.
🔸 [True] 5.23. <시간> <지역> <건물> <건물> 지하주차장 화재로 인해 차량통제 중임을 알려드립니다.

🔹 [Pred] <시간> <지역>K통신구에 발생하였으니, 주민들께서는 창문을 안전에 바랍니다..
🔸 [True] 5.23. <시간> <지역>

[Epoch 20] Training: 100%|██████████| 2487/2487 [02:14<00:00, 18.49it/s, train_loss=0.244]



🟦 [Train Samples]
🔹 [Pred] <지역> 658번지 인근 <건물> 화재가 발생하였으니 남산로 일대 차량통행을 자제하시기 바랍니다.
🔸 [True] 현재 <지역> 658번지 인근 <건물> 화재가 발생하였으니 남산로 일대 차량통행을 자제하시기 바랍니다.

🔹 [Pred] <시간> <지역> <주소> 화재 발생. 인근 주민은 대피 및 차량운행을 자제하기 바랍니다
🔸 [True] 오늘 <시간> <지역> <주소> 화재 발생. 인근 주민은 대피 및 차량운행을 자제하기 바랍니다.

🔹 [Pred] .24 <시간> <지역> <지역> 56번길 화재 발생. 인근 주민은 안전한 곳으로 대피하여 주시기 바랍니다.
🔸 [True] 4.24 <시간> <지역> <지역> 56번길 화재 발생. 인근 주민은 안전한 곳으로 대피하여 주시기 바랍니다.

🔹 [Pred] <시간> <지역> <도로> 양구리구간 화재 발생, 인근 주민은 안전한 곳으로 대피 바랍니다.
🔸 [True] 오늘 <시간> <지역> <도로> 양구리구간 화재 발생, 인근 주민은 안전한 곳으로 대피 바랍니다.

🔹 [Pred] .29 <시간> 부로 <지역> <주소> 화재 발생, 인근 주민은 안전한 곳으로 대피 바람
🔸 [True] 7.29 <시간> 부로 <지역> <주소> 화재 발생, 인근 주민은 안전한 곳으로 대피 바람


🟩 [Validation Samples]
🔹 [Pred] <시간> 발생한<역>에 화재 발생 발생 인근 주민은 안전에 유의 바랍니다.
🔸 [True] 5.23. <시간> <지역> <건물> <건물> 지하주차장 화재로 인해 차량통제 중임을 알려드립니다.

🔹 [Pred] <시간> <지역> <주소> 화재 발생. 인근 주민은<역> 지역을 우회하여 주시기 바랍니다.
🔸 [True] 5.23. <시간> <지역> <건물> <건물> 지하주차장 화재로 인해 차량통제 중임을 알려드립니다.

🔹 [Pred] <시간> <역>에 화재가 발생하였으니, 인근 주민은 안전에 유의 바랍니다
🔸 [True] 5.23. <시간> <

In [27]:
# Tokenization 예

sp = spm.SentencePieceProcessor()
sp.load("training/train/processed/spm.model")

print(sp.encode("주시기 바랍니다", out_type=str))
print(sp.encode("양해 바랍니다", out_type=str))
print(sp.encode("화재 발생으로 인해", out_type=str))


['▁주시기', '▁바랍니다']
['▁양해', '▁바랍니다']
['▁화재', '▁발생으로', '▁인해']


In [8]:
from tqdm import tqdm
import pandas as pd

def evaluate_validation_set(model, val_loader, tokenizer, device, eos_id=2, pad_id=3):
    model.eval()
    all_preds = []
    all_refs = []

    with torch.no_grad():
        for X_val, y_val in tqdm(val_loader, desc="Evaluating"):
            X_val, y_val = X_val.to(device), y_val.to(device)

            # 🔹 디코딩된 예측 (sampling 기반)
            preds = [decode_sequence(model, x, tokenizer, device, eos_id=eos_id) for x in X_val]

            # 🔹 정답 시퀀스를 디코딩 (token → string)
            refs = decode_tokens(tokenizer, y_val.tolist(), eos_id=eos_id, pad_id=pad_id)

            all_preds.extend(preds)
            all_refs.extend(refs)

    # 🔹 결과 정리
    return pd.DataFrame({
        "prediction": all_preds,
        "reference": all_refs
    })


In [30]:
# 1. 모델 로드 
model = Seq2Seq(encoder, decoder, device).to(device)

checkpoint = torch.load("checkpoints/best_checkpoint.pt", map_location=device, weights_only=False)
model.load_state_dict(checkpoint["model_state_dict"])
model = model.to(device)
model.eval()

# 2. Validation loader 준비 (이미 했다면 생략)
val_loader = get_validation_loader("/teamspace/studios/this_studio/training/val/processed")

# 3. 평가 및 디코딩
df_result = evaluate_validation_set(model, val_loader, spm_tokenizer, device)

# 5. 저장 (선택)
df_result.to_csv("val_predictions.csv", index=False, encoding="utf-8-sig")

NameError: name 'evaluate_validation_set' is not defined

In [7]:
!ls checkpoints

best_checkpoint.pt     checkpoint_epoch16.pt  checkpoint_epoch23.pt
checkpoint_epoch1.pt   checkpoint_epoch17.pt  checkpoint_epoch3.pt
checkpoint_epoch10.pt  checkpoint_epoch18.pt  checkpoint_epoch4.pt
checkpoint_epoch11.pt  checkpoint_epoch19.pt  checkpoint_epoch5.pt
checkpoint_epoch12.pt  checkpoint_epoch2.pt   checkpoint_epoch6.pt
checkpoint_epoch13.pt  checkpoint_epoch20.pt  checkpoint_epoch7.pt
checkpoint_epoch14.pt  checkpoint_epoch21.pt  checkpoint_epoch8.pt
checkpoint_epoch15.pt  checkpoint_epoch22.pt  checkpoint_epoch9.pt


In [54]:
# Checkpoint 비워주기

import glob
checkpoint_dir = "checkpoints"

# 해당 폴더 안의 모든 파일 삭제
files = glob.glob(os.path.join(checkpoint_dir, "*"))
for f in files:
    os.remove(f)

print(f"✅ {checkpoint_dir} 폴더 안 모든 파일이 삭제되었습니다.")

✅ checkpoints 폴더 안 모든 파일이 삭제되었습니다.


In [8]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
from tqdm import tqdm

def restore_slots(masked_text: str, slot_values: dict) -> str:
    restored = masked_text
    for slot, value in slot_values.items():
        restored = restored.replace(slot, value)
    return restored

def restore_pred_and_ref_with_file_ids(pred_csv_path: str, json_dir: str, file_id_npy_path: str):
    # 1. Load prediction DataFrame
    df = pd.read_csv(pred_csv_path)

    # 2. Load file IDs from .npy
    file_ids = np.load(file_id_npy_path)

    # 3. Iterate and restore both prediction and reference
    restored_preds = []
    restored_refs = []

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        file_id = file_ids[idx]
        json_path = Path(json_dir) / f"{file_id}.json"

        if json_path.exists():
            with open(json_path, encoding="utf-8") as f:
                data = json.load(f)
            slot_values = data.get("slot_values", {})
        else:
            slot_values = {}

        pred_restored = restore_slots(row["prediction"], slot_values)
        ref_restored = restore_slots(row["reference"], slot_values)

        restored_preds.append(pred_restored)
        restored_refs.append(ref_restored)

    # 4. Add new columns to DataFrame
    df["restored_prediction"] = restored_preds
    df["restored_reference"] = restored_refs
    df["file_id"] = file_ids  # ID까지 붙여주면 디버깅 편함
    return df


In [9]:
df_restored = restore_pred_and_ref_with_file_ids(
    pred_csv_path="val_predictions.csv",
    json_dir="training/val/norm_keypoint",
    file_id_npy_path="training/val/processed/file_id_part_0.npy"
)

# 미리보기
print(df_restored[["file_id", "prediction", "reference", "restored_prediction", "restored_reference"]].head())

# 저장
df_restored.to_csv("val_predictions_restored.csv", index=False, encoding="utf-8-sig")


100%|██████████| 408/408 [00:08<00:00, 48.44it/s]

                              file_id  \
0  NIA_SL_G2_FIRE000004_1_KU02_F_norm   
1  NIA_SL_G2_FIRE000004_2_KU02_F_norm   
2  NIA_SL_G2_FIRE000004_3_KU02_F_norm   
3  NIA_SL_G2_FIRE000020_1_KU02_F_norm   
4  NIA_SL_G2_FIRE000020_2_KU02_F_norm   

                                          prediction  \
0  <도로>  대피하시기 주민들께서는 지역을 대피하기<도로> 화재 발생으로 인근에 분들...   
1                  <지역> <도로>에서 발생으로 일부 차량통제,으로 바랍니다.   
2            <지역> <주소> 화재 발생으로 인근 주민은<도로>의지하오니 바랍니다.   
3              <시간> <지역> <건물> 화재발생, 인근 에 안전에 유의 바랍니다   
4     <시간> <지역> <주소>에서 화재발생, 인근<건물> 주민들은 안전에 유의 바랍니다   

                                           reference  \
0  5.23. <시간> <지역> <건물> <건물> 지하주차장 화재로 인해 차량통제 중임...   
1  5.23. <시간> <지역> <건물> <건물> 지하주차장 화재로 인해 차량통제 중임...   
2  5.23. <시간> <지역> <건물> <건물> 지하주차장 화재로 인해 차량통제 중임...   
3  오늘 <시간> 반월공단 내 대림비앤코 화재로 인해 차량통행 불가하오니 우회하여 주시...   
4  오늘 <시간> 반월공단 내 대림비앤코 화재로 인해 차량통행 불가하오니 우회하여 주시...   

                                 restored_prediction  \
0  <도로>  대피하시기 주민들께서는 지

In [11]:
# 시연 
# keypoint 추출

import cv2
import mediapipe as mp
import numpy as np
from tqdm import tqdm

# MediaPipe pose 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False)
mp_drawing = mp.solutions.drawing_utils

def extract_keypoints_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    all_keypoints = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # BGR → RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Pose 추출
        pose_result = pose.process(image)
        hand_result = hands.process(image)

        keypoints = []

        # Pose keypoints
        if pose_result.pose_landmarks:
            for lm in pose_result.pose_landmarks.landmark:
                keypoints.extend([lm.x, lm.y, lm.visibility])
        else:
            keypoints.extend([0.0, 0.0, 0.0] * 33)

        # Hands keypoints (왼손 + 오른손)
        for hand_landmarks in [hand_result.left_hand_landmarks, hand_result.right_hand_landmarks]:
            if hand_landmarks:
                for lm in hand_landmarks.landmark:
                    keypoints.extend([lm.x, lm.y, lm.visibility])
            else:
                keypoints.extend([0.0, 0.0, 0.0] * 21)

        # 총 keypoint 수 확인 (33 + 21 + 21 = 75개 → 75 x 3 = 225차원)
        all_keypoints.append(keypoints)

    cap.release()
    return np.array(all_keypoints)  # shape: (T, 225)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1751285248.457784   11380 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751285248.474995   11382 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751285248.487133   11377 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751285248.523309   11379 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [19]:
import cv2
import mediapipe as mp
import numpy as np
import os
from tqdm import tqdm

# 🎯 유지할 pose index
POSE_KEEP_IDX = [0, 1, 2, 3, 4, 5, 6, 7, 15, 16, 17, 18]

def extract_video_frames(video_path, fps=30):
    cap = cv2.VideoCapture(video_path)
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(original_fps // fps) if original_fps > fps else 1

    frames = []
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_interval == 0:
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        count += 1
    cap.release()
    return frames

def extract_filtered_keypoints(frames):
    mp_pose = mp.solutions.pose
    mp_hands = mp.solutions.hands

    pose = mp_pose.Pose(static_image_mode=False)
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2)

    results = []
    for frame in tqdm(frames, desc="📌 MediaPipe 처리 중"):
        frame_kpts = []

        # 🔹 Pose keypoints
        pose_result = pose.process(frame)
        if pose_result.pose_landmarks:
            pose_kpts = pose_result.pose_landmarks.landmark
            for idx in POSE_KEEP_IDX:
                lm = pose_kpts[idx]
                frame_kpts.extend([lm.x, lm.y])
        else:
            frame_kpts.extend([0.0, 0.0] * len(POSE_KEEP_IDX))

        # 🔹 Hands keypoints
        hand_result = hands.process(frame)
        left_hand = [0.0, 0.0] * 21
        right_hand = [0.0, 0.0] * 21

        if hand_result.multi_hand_landmarks and hand_result.multi_handedness:
            for i, hand_landmarks in enumerate(hand_result.multi_hand_landmarks):
                label = hand_result.multi_handedness[i].classification[0].label
                coords = [coord for lm in hand_landmarks.landmark for coord in (lm.x, lm.y)]
                if label == "Left":
                    left_hand = coords
                elif label == "Right":
                    right_hand = coords

        frame_kpts.extend(left_hand)
        frame_kpts.extend(right_hand)
        results.append(frame_kpts)

    return np.array(results)  # shape: (T, 108)

# ✅ 실행 예시
video_path = "video.mp4"

frames = extract_video_frames(video_path, fps=30)
kpt_108 = extract_filtered_keypoints(frames)

print("▶ 최종 shape:", kpt_108.shape)  # (T, 108)


📌 MediaPipe 처리 중:   0%|          | 0/707 [00:00<?, ?it/s]W0000 00:00:1751286223.754911   14048 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751286223.773966   14044 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751286223.777339   14048 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751286223.813469   14044 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
📌 MediaPipe 처리 중: 100%|██████████| 707/707 [00:29<00:00, 23.76it/s]

▶ 최종 shape: (707, 108)


In [21]:
# Size 복원 후 정규화 

import numpy as np

def normalize_mediapipe_keypoints(kpt_108: np.ndarray, stat_path: str,
                                   image_width=1280, image_height=720) -> np.ndarray:
    """
    Normalize MediaPipe keypoints using OpenPose-based statistics.
    
    Args:
        kpt_108 (np.ndarray): Keypoints of shape (T, 108) with (x, y) in range [0, 1]
        stat_path (str): Path to 'norm_stat.npz' from training
        image_width (int): Width of original video frames
        image_height (int): Height of original video frames
        
    Returns:
        np.ndarray: Normalized keypoints of shape (T, 108)
    """
    # 1. 좌표 복원 (0~1 → 픽셀)
    kpt = np.copy(kpt_108)
    kpt[:, 0::2] *= image_width   # x 좌표
    kpt[:, 1::2] *= image_height  # y 좌표

    # 2. 통계 불러오기
    stats = np.load(stat_path)
    pose_mu = stats["pose_mu"]
    pose_sd = stats["pose_sd"]
    left_min = stats["left_min"]
    left_max = stats["left_max"]
    right_min = stats["right_min"]
    right_max = stats["right_max"]

    # 3. 분할
    pose = kpt[:, :24]
    left = kpt[:, 24:66]
    right = kpt[:, 66:]

    # 4. 정규화
    norm_pose = (pose - pose_mu) / (pose_sd + 1e-8)
    norm_left = (left - left_min) / (left_max - left_min + 1e-8) - 0.5
    norm_right = (right - right_min) / (right_max - right_min + 1e-8) - 0.5

    # 5. 합치기
    norm_kpt = np.concatenate([norm_pose, norm_left, norm_right], axis=-1)
    return norm_kpt

norm_kpt = normalize_mediapipe_keypoints(kpt_108, stat_path="training/norm_stat.npz")
print("▶ 정규화된 shape:", norm_kpt.shape)  # (T, 108)


▶ 정규화된 shape: (707, 108)


In [22]:
import numpy as np

# Pose (앞쪽 24차원): 12개 keypoint × 2 = 24
pose = norm_kpt[:, :24]
# Left hand (중간 42차원): 21개 keypoint × 2 = 42
left = norm_kpt[:, 24:66]
# Right hand (뒤쪽 42차원)
right = norm_kpt[:, 66:]

print("✅ Pose 평균과 표준편차")
print("Mean:", pose.mean(axis=0))
print("Std: ", pose.std(axis=0))

print("\n✅ Left hand 범위")
print("Min:", left.min(), "Max:", left.max())

print("\n✅ Right hand 범위")
print("Min:", right.min(), "Max:", right.max())


✅ Pose 평균과 표준편차
Mean: [-11.16571178   1.61261149 -12.24069937  -5.13413468  -4.6927129
  -4.9362072   -1.33633479  -4.47444851  -2.59699363  -1.37483823
 -21.97698555  -4.95795615 -13.19388483  -5.35643645  -6.284303
  -1.91416178  -5.98963323   6.05604604 -15.13240997   5.09070572
  -3.08469361   5.56407687 -10.08432758   4.61333059]
Std:  [0.19218931 0.06411243 0.18897083 0.0786324  0.13019507 0.07248655
 0.06903451 0.02829407 0.06687626 0.01036941 0.21694626 0.0653447
 0.10574796 0.02625473 0.05032789 0.00819363 1.37046651 1.27776483
 1.44009533 0.94750438 1.15802779 1.67799417 1.16173833 1.25872799]

✅ Left hand 범위
Min: -0.5 Max: 0.061749881855847955

✅ Right hand 범위
Min: -0.5 Max: 0.20890637965029302


In [23]:
import torch

def predict_from_keypoint(model, keypoint_seq, tokenizer, device, eos_id=2, max_len=100):
    model.eval()

    # 1. numpy → torch tensor 변환: (T, 108) → (1, T, 108)
    x = torch.tensor(keypoint_seq, dtype=torch.float32).unsqueeze(0).to(device)

    # 2. Decoder 시작 토큰: BOS
    bos_id = tokenizer.bos_id()
    decoder_input = torch.tensor([[bos_id]], dtype=torch.long).to(device)

    preds = []

    with torch.no_grad():
        for _ in range(max_len):
            # 3. 모델 forward
            output = model(x, decoder_input, teacher_forcing_ratio=0.0)

            # 4. 가장 마지막 토큰의 확률 → 예측 token id
            next_token = output[:, -1, :].argmax(dim=-1).item()

            # 5. EOS 토큰이면 종료
            if next_token == eos_id:
                break

            preds.append(next_token)

            # 6. decoder input 업데이트
            next_token_tensor = torch.tensor([[next_token]], dtype=torch.long).to(device)
            decoder_input = torch.cat([decoder_input, next_token_tensor], dim=1)

    # 7. 토큰 ID → 문장 복원
    return tokenizer.decode(preds)


In [31]:
# 1. 모델 로드 
model = Seq2Seq(encoder, decoder, device).to(device)

checkpoint = torch.load("checkpoints/best_checkpoint.pt", map_location=device, weights_only=False)
model.load_state_dict(checkpoint["model_state_dict"])
model = model.to(device)
model.eval()

Seq2Seq(
  (encoder): Encoder(
    (rnn): GRU(108, 256, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(1000, 256)
    (attention): Attention(
      (attn): Linear(in_features=768, out_features=256, bias=True)
      (v): Linear(in_features=256, out_features=1, bias=False)
    )
    (rnn): GRU(768, 256, batch_first=True)
    (fc_out): Linear(in_features=1024, out_features=1000, bias=True)
  )
)

In [32]:
# 예측
output_text = predict_from_keypoint(model, norm_kpt, spm_tokenizer, device)
print("🔍 예측 결과:", output_text)


🔍 예측 결과:  ⁇ <시간> 발생으로<도로><도로><도로> 발생으로 발생으로<도로><도로> 일대가 일대가 대피 일대 대피 발생으로 대피 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대가 일대


In [46]:
# 추출이 잘 안됨

import numpy as np
import pandas as pd

# Convert the first few frames to a DataFrame for easier inspection
df = pd.DataFrame(kpt_108)  # Show first 5 frames

total_elements = df.size

# 0의 개수
zero_count = (df < 0.1).sum().sum()

# 비율 계산 (%)
zero_ratio = (zero_count / total_elements) * 100

print(f"총 값 중 0의 비율: {zero_ratio:.2f}%")

총 값 중 0의 비율: 14.47%
